In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import subprocess

# Define the folder where latest .whl file exists
FOLDER = "gs://agent_framework/latest/"

# Execute gsutil command to list files in the folder
command_output = subprocess.check_output(['gsutil', 'ls', FOLDER]).decode('utf-8')
file_list = command_output.strip().split('\n')

# Find the .whl file
whl_file = next((file for file in file_list if file.endswith('.whl')), None)

if whl_file:
    RELEASE_VERSION = whl_file.split('/')[-1]
    print(f"RELEASE_VERSION: {RELEASE_VERSION}")
else:
    print("No .whl file found!")

!gsutil cp {FOLDER}{RELEASE_VERSION} .
!pip3 install {RELEASE_VERSION} --quiet

RELEASE_VERSION: google_adk-0.0.2.dev20250326+nightly740999296-py3-none-any.whl
Copying gs://agent_framework/latest/google_adk-0.0.2.dev20250326+nightly740999296-py3-none-any.whl...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.4/231.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # Make sure you have .whl downlaoded from:
# # https://drive.google.com/file/d/1WtfjKtbV66aiZP_k0ncZoY-4_ON4sP7Q/view?usp=drive_link
# !pip3 install google_adk-0.0.2.dev20250324+739344859-py3-none-any.whl

`Make sure to restart the session`

In [ ]:
import os

PROJECT_ID = "lavi-llm-experiment"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

# If you want to run Antropic Models, make sure the location is:
# us-east5 or europe-west1

In [ ]:
# # Temp - to supress WARNING:google_genai.types:
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [ ]:
GEMINI_2_FLASH = "gemini-2.0-flash-001"

## LLMAgent/Agent

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import built_in_google_search  # Import the tool

root_agent = Agent(
   # A unique name for the agent.
   name="basic_search_agent",
   # The Large Language Model (LLM) that agent will use.
   model="gemini-2.0-flash-001",
   # A short description of the agent's purpose.
   description="Agent to answer questions using Google Search.",
   # Instructions to set the agent's behavior.
   instruction="You are an expert researcher. You always stick to the facts.",
   # Add google_search tool to perform grounding with Google search.
   tools=[built_in_google_search]
)


In [ ]:
from google.adk.agents import Agent
from google.adk.tools import built_in_google_search  # Import the tool

root_agent = Agent(
   # A unique name for the agent.
   name="basic_search_agent",
   # The Large Language Model (LLM) that agent will use.
   model="gemini-2.0-flash-001",
   # A short description of the agent's purpose.
   description="Agent to answer questions using Google Search.",
   # Instructions to set the agent's behavior.
   instruction="You are an expert researcher. You always stick to the facts.",
   # Add google_search tool to perform grounding with Google search.
   tools=[built_in_google_search]
)


### LLM Agent with a Single Tool

In [ ]:
from google.adk.agents import LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# Constant
APP_NAME = "capital_city_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "capital_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"


# Define a simple tool
def get_capital_city(country: str) -> str:
    """Retrieves the capital city of a given country.

    Args:
        country: The name of the country.

    Returns:
        The capital city of the country.
    """
    country_capitals = {
        "united states": "washington, d.c.",
        "canada": "ottawa",
        "france": "paris",
    }
    return country_capitals.get(country.lower(), "Capital not found")


# Agent
capital_agent = LlmAgent(
    model="gemini-2.0-flash-001",
    name="capital_agent",
    description="An agent that can retrieve the capital city of a country.",
    instruction="""You are an agent that can retrieve the capital city of a country.
    When a user provides a prompt, extract the country name.
    Then, use the `get_capital_city` tool to retrieve the capital city for that country.
    Finally, present the capital city to the user in a clear and concise manner.
    """,
    tools=[get_capital_city],
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=capital_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("What is the capital of france?")

Agent Response:  The capital of France is Paris.



### Simple LLM Agent

In [ ]:
from google.adk.agents import LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

# Constant
APP_NAME = "capital_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "capital_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# Agent
capital_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Capital Information Agent. Your task is to provide the capital of a given city.

    When a user provides a prompt, extract the city name.
    Then, respond with the capital of that city.
    """,
    description="""You are an agent who can tell the capital of a city.""",
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=capital_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("What's the capital of France?")
call_agent("What's the capital of Germany?")
call_agent("What's the capital of Japan?")

Agent Response:  The capital of France is Paris.

Agent Response:  The capital of Germany is Berlin.

Agent Response:  The capital of Japan is Tokyo.



### LLM Agent with a Input/Output Schema

In [ ]:
from pydantic import BaseModel
from pydantic import Field
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent

# Constant
APP_NAME = "capital_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "capital_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"


class InputSchema(BaseModel):
    country: str = Field(description="The country to find the capital of.")


class OutputSchema(BaseModel):
    capital: str = Field(description="The capital of the country.")


# Agent
capital_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Capital Information Agent. Your task is to provide the capital of a given country.

    When a user provides a prompt, extract the country name.
    Then, respond with the capital of that country in the following JSON format:
    {"capital": "capital_name"}
    """,
    description="""You are an agent who can tell the capital of a country.""",
    allow_transfer=False,
    input_schema=InputSchema,
    output_schema=OutputSchema,
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)
runner = Runner(
    agent=capital_agent, app_name=APP_NAME, session_service=session_service
)


# Agent Interaction
def call_agent(query):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id=USER_ID, session_id=SESSION_ID, new_message=content
    )

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

call_agent('{"country": "France"}')
call_agent('{"country": "Germany"}')
call_agent('{"country": "Japan"}')

Agent Response:  {
"capital": "Paris"
}
Agent Response:  {
"capital": "Berlin"
}
Agent Response:  {
"capital": "Tokyo"
}


### LLM Agent with a output_key

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
import json


# Constant
APP_NAME = "capital_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "capital_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# Agent
capital_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Capital Information Agent. Your task is to provide the capital of a given country.

    When a user provides a prompt, extract the country name.
    Then, respond with the capital of that country in the following JSON format:
    {"capital": "capital_name"}
    """,
    description="""You are an agent who can tell the capital of a country.""",
    output_key="capital_output",
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)
runner = Runner(
    agent=capital_agent, app_name=APP_NAME, session_service=session_service
)


# Agent Interaction
def call_agent(query):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id=USER_ID, session_id=SESSION_ID, new_message=content
    )

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)
            print(
                "Session State:",
                session_service.get_session(
                    APP_NAME, USER_ID, SESSION_ID
                ).state.get("capital_output"),
            )


call_agent("What's the capital of France?")
call_agent("What's the capital of Germany?")
call_agent("What's the capital of Japan?")

Agent Response:  ```json
{
"capital": "Paris"
}
```
Session State: ```json
{
"capital": "Paris"
}
```
Agent Response:  ```json
{
"capital": "Berlin"
}
```
Session State: ```json
{
"capital": "Berlin"
}
```
Agent Response:  ```json
{
"capital": "Tokyo"
}
```
Session State: ```json
{
"capital": "Tokyo"
}
```


### LLM Agent with a built_in_code_execution

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
import json
from google.adk.tools import built_in_code_execution

# Constant
APP_NAME = "capital_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "capital_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

code_agent = LlmAgent(
        name="code_execution_agent",
        model=GEMINI_2_FLASH,
        tools=[built_in_code_execution],
        instruction="Generate python code to solve the user's request. "
        "If the user asks for a specific output, return the output of the code execution.",
    )
# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=code_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("Write a python function to calculate the factorial of a number and return the result")

Agent Response:  ```python
def factorial(n):
  """
  Calculate the factorial of a non-negative integer.

  Args:
    n: A non-negative integer.

  Returns:
    The factorial of n, which is the product of all positive integers less than or equal to n.
    Returns 1 if n is 0.
    Raises ValueError if n is negative.
  """
  if n < 0:
    raise ValueError("Factorial is not defined for negative numbers")
  elif n == 0:
    return 1
  else:
    result = 1
    for i in range(1, n + 1):
      result *= i
    return result
```


### LLM Agent with a Single Tool

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent

# Constant
APP_NAME = "weather_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "weather_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},  # Fixed typo: 'tempeerature' to 'temperature'
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."

# Agent
weather_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Weather Information Agent. Your task is to provide weather information for a given city.

    When a user provides a prompt, extract the city name.
    Then, use the `get_weather` tool to retrieve the weather information for that city.
    Finally, present the weather information to the user in a clear and concise manner.""",
    description="""You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task.""",
    tools=[get_weather],
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=weather_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("What's the weather in toronto?")

Agent Response:  OK. The weather in Toronto is 30 degrees Celsius, partly cloudy with an overcast sky.



### LLM Agent with a Multiple Tool

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent

# Constants
APP_NAME = "recipe_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "recipe_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# --- Mock Data ---
recipes = {
    "pasta carbonara": {
        "ingredients": ["pasta", "eggs", "guanciale", "pecorino romano", "black pepper"],
        "dietary_restrictions": ["none"],
    },
    "chicken tikka masala": {
        "ingredients": ["chicken", "yogurt", "ginger", "garlic", "masala blend"],
        "dietary_restrictions": ["none"],
    },
    "vegan lentil soup": {
        "ingredients": ["lentils", "carrots", "celery", "onion", "vegetable broth"],
        "dietary_restrictions": ["vegan"],
    },
}


# --- Tools ---
def search_recipes(keyword: str) -> str:
    """Searches for recipes based on a keyword.

    Args:
        keyword: The keyword to search for (e.g., ingredient).

    Returns:
        A string containing the names of matching recipes, or a message if no recipes are found.
    """
    matching_recipes = [
        recipe_name
        for recipe_name, recipe_data in recipes.items()
        if keyword.lower() in recipe_name.lower() or keyword.lower() in recipe_data["ingredients"]
    ]
    if matching_recipes:
        return f"Recipes matching '{keyword}': {', '.join(matching_recipes)}."
    else:
        return f"No recipes found matching '{keyword}'."


def check_dietary_restrictions(recipe_name: str, dietary_restriction: str) -> str:
    """Checks if a recipe is suitable for a given dietary restriction.

    Args:
        recipe_name: The name of the recipe to check.
        dietary_restriction: The dietary restriction to check for (e.g., "vegan").

    Returns:
        A string indicating if the recipe is suitable or not.
    """
    recipe_data = recipes.get(recipe_name.lower())
    if recipe_data:
        if dietary_restriction.lower() in recipe_data["dietary_restrictions"]:
            return f"'{recipe_name}' is suitable for a '{dietary_restriction}' diet."
        else:
            return f"'{recipe_name}' is not suitable for a '{dietary_restriction}' diet."
    else:
        return f"Recipe '{recipe_name}' not found."


def get_ingredient_list(recipe_name: str) -> str:
    """Returns a list of ingredients for a given recipe.

    Args:
        recipe_name: The name of the recipe.

    Returns:
        A string containing the list of ingredients, or a message if the recipe is not found.
    """
    recipe_data = recipes.get(recipe_name.lower())
    if recipe_data:
        return f"Ingredients for '{recipe_name}': {', '.join(recipe_data['ingredients'])}."
    else:
        return f"Recipe '{recipe_name}' not found."


# --- Agent ---
recipe_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Recipe Agent. Your task is to help users find recipes and check their suitability for dietary restrictions.

    You have access to three tools:
    1. `search_recipes`: Use this tool to find recipes based on a keyword (e.g., ingredient).
    2. `check_dietary_restrictions`: Use this tool to check if a recipe is suitable for a given dietary restriction.
    3. `get_ingredient_list`: Use this tool to get a list of ingredients for a given recipe.

    When a user provides a prompt, first determine what they are asking for.
    - If they are asking for recipes based on a keyword, use the `search_recipes` tool.
    - If they are asking if a recipe is suitable for a dietary restriction, use the `check_dietary_restrictions` tool.
    - If they are asking for a list of ingredients for a recipe, use the `get_ingredient_list` tool.
    Finally, present the information to the user in a clear and concise manner.
    """,
    description="""An agent that can find recipes, check dietary restrictions, and list ingredients.
    It has access to the `search_recipes`, `check_dietary_restrictions`, and `get_ingredient_list` tools.""",
    tools=[search_recipes, check_dietary_restrictions, get_ingredient_list],
)

# --- Session and Runner ---
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=recipe_agent, app_name=APP_NAME, session_service=session_service)


# --- Agent Interaction ---
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)


call_agent("Find recipes with chicken.")
call_agent("Is pasta carbonara suitable for a vegan diet?")
call_agent("What are the ingredients in vegan lentil soup?")

Agent Response:  I found one recipe matching your request: chicken tikka masala.


Agent Response:  No, pasta carbonara is not suitable for a vegan diet.


Agent Response:  The ingredients for vegan lentil soup are: lentils, carrots, celery, onion, vegetable broth.



### LLM Agent with a Async Run (run_async)

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent


# Constant
APP_NAME = "weather_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "weather_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},  # Fixed typo: 'tempeerature' to 'temperature'
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."

# Agent
weather_agent = Agent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Weather Information Agent. Your task is to provide weather information for a given city.

    When a user provides a prompt, extract the city name.
    Then, use the `get_weather` tool to retrieve the weather information for that city.
    Finally, present the weather information to the user in a clear and concise manner.""",
    description="""You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task.""",
    tools=[get_weather],
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=weather_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

await call_agent_async("What's the weather in toronto?")

Agent Response:  OK. The weather in Toronto is 30 degrees Celsius, partly cloudy with an overcast sky.



### LLM Agent with Auto Flow (using LLM)

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
import asyncio

# Constant
APP_NAME = "weather_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "weather_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"


# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."


def get_greeting(name: str) -> str:
    """Greets the given name.

    Args:
        name: The name to greet.

    Returns:
        A greeting message.
    """
    return f"Hello, {name}!"


# Agent
weather_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name="weather_agent",
    instruction="""You are a Weather Information Agent. Your task is to provide weather information for a given city.

    When a user provides a prompt, extract the city name.
    Then, use the `get_weather` tool to retrieve the weather information for that city.
    Finally, present the weather information to the user in a clear and concise manner.
    If the user asks for a greeting, transfer to the greeting agent.""",
    description="""You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task.""",
    tools=[get_weather],
    allow_transfer=True,
)

greeting_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name="greeting_agent",
    instruction="""You are a Greeting Agent. Your task is to greet the user.

    When a user provides a prompt, extract the name.
    Then, use the `get_greeting` tool to greet the user.
    Finally, present the greeting to the user in a clear and concise manner.
    If the user asks for weather information, transfer to the weather agent.""",
    description="""You are an agent who can greet a user.
    You have access to the `get_greeting` tool to accomplish this task.""",
    tools=[get_greeting],
    allow_transfer=True,
)

# Set parent-child relationship
weather_agent.children = [greeting_agent]
# greeting_agent.parent_agent = weather_agent

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=weather_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
async def call_agent_async(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)


In [ ]:
await call_agent_async("What's the weather in toronto?")

Agent Response:  The weather in Toronto is 30 degrees Celsius, partly cloudy with an overcast sky.



In [ ]:
await call_agent_async("Hello, Alice!")

Agent Response:  Hello, Alice!



### Weather Agent QUickstart with Multi-Agent

In [ ]:
# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."


# Tool for the Greeting Agent
def say_hello(name: str = "there") -> str:
    """Provides a simple greeting."""
    return f"Hello, {name}!"

# Tool for the Farewell Agent
def say_goodbye() -> str:
    """Provides a simple farewell message."""
    return "Goodbye! Have a great day."

In [ ]:
# --- Import LlmAgent ---
from google.adk.agents import LlmAgent
import google.genai.types as types # For Content/Part later

# --- Agent Definitions ---
AGENT_NAME_WEATHER = "weather_agent"
AGENT_NAME_GREETING = "greeting_agent"
AGENT_NAME_FAREWELL = "farewell_agent"
MODEL_NAME = "gemini-2.0-flash-001" # Use a recent flash model

# --- Parent Agent: Weather ---
weather_agent = LlmAgent(
    model=MODEL_NAME,
    name=AGENT_NAME_WEATHER,
    # *** REVISED INSTRUCTION ***
    instruction=f"""You are the main Weather Agent in charge. Your primary responsibility is providing weather information.
    - **IF** the user asks specifically about the weather (e.g., 'weather in city', 'forecast'), use the 'get_weather' tool YOURSELF. **DO NOT transfer weather requests.**
    - **ONLY IF** the user gives a simple greeting (like 'Hi', 'Hello') with NO other request, transfer to the '{AGENT_NAME_GREETING}'.
    - **ONLY IF** the user explicitly says goodbye (like 'Bye', 'See you'), transfer to the '{AGENT_NAME_FAREWELL}'.
    - Handle only weather requests directly.
    """,
    description="Provides weather forecasts using 'get_weather'. Delegates greetings/farewells.",
    tools=[get_weather],
    allow_transfer=True, # Still needed to enable delegation *when appropriate*
)
print(f"Defined parent agent: {weather_agent.name}")


# --- Child Agent 1: Greeting ---
greeting_agent = LlmAgent(
    model=MODEL_NAME,
    name=AGENT_NAME_GREETING,
    instruction="You are the Greeting Agent. Use the 'say_hello' tool to greet the user. Do nothing else.",
    description="Handles simple greetings using the 'say_hello' tool.",
    tools=[say_hello],
    allow_transfer=True,

)

# --- Child Agent 2: Farewell ---
farewell_agent = LlmAgent(
    model=MODEL_NAME,
    name=AGENT_NAME_FAREWELL,
    instruction="You are the Farewell Agent. Use the 'say_goodbye' tool when the user indicates they are leaving. Do nothing else.",
    description="Handles simple farewells using the 'say_goodbye' tool.",
    tools=[say_goodbye],
    allow_transfer=True,
)

# --- Define the Parent-Child Relationship ---
# This tells the framework how the agents are structured.
weather_agent.children = [greeting_agent, farewell_agent]
# The framework automatically sets the .parent_agent attribute on the children

print("Defined weather_agent (parent), greeting_agent (child), and farewell_agent (child).")
print(f"{weather_agent.name} children: {[child.name for child in weather_agent.children]}")

Defined parent agent: weather_agent
Defined weather_agent (parent), greeting_agent (child), and farewell_agent (child).
weather_agent children: ['greeting_agent', 'farewell_agent']


In [ ]:
from google.adk.sessions import InMemorySessionService
import uuid

session_service = InMemorySessionService()

# Required. Unique identifier for the application.
APP_NAME = "weather_app"
# Required. Identifier for the user interacting with the agent. This is a dynamic variable.
USER_ID = "12345"

SESSION_ID = f"session_{uuid.uuid4()}" # Use a dynamic session ID
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

In [ ]:
from google.adk.runners import Runner

runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service,
)


In [ ]:
def call_agent(user_query):
  content = types.Content(role='user', parts=[types.Part(text=user_query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)
  return None

In [ ]:
call_agent(user_query = "Hello there!")

Agent Response:  Hello, there!



In [ ]:
call_agent(user_query = "What's the weather like in Chennai?")

Agent Response:  Sorry, I am designed to only greet people. I cannot help you with the weather.



In [ ]:
call_agent(user_query = "What about the weather in toronto?")

Agent Response:  I am here to greet you. I don't have the ability to look up weather information.



In [ ]:
call_agent(user_query = "Okay, thanks, bye!")

Agent Response:  Goodbye!



In [ ]:
from google.adk.sessions import InMemorySessionService
import uuid

# Required. Unique identifier for the application.
APP_NAME = "multi_weather_app"
# Required. Identifier for the user interacting with the agent.
USER_ID = "user_xyz"
SESSION_ID = f"session_{uuid.uuid4()}" # Use a dynamic session ID

session_service = InMemorySessionService()
session = session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

print(f"Session Service setup complete. Session ID: {session.id}")

Session Service setup complete. Session ID: session_ba542e33-3ec3-4a85-8c69-9d0a3544327a


In [ ]:
from google.adk.runners import Runner

runner = Runner(
    agent=weather_agent, # Start with the main weather agent
    app_name=APP_NAME,
    session_service=session_service,
    # artifact_service=... # Add if using artifacts
)

print(f"Runner created, configured with root agent: {runner.agent.name}")

Runner created, configured with root agent: weather_agent


In [ ]:
import asyncio # Needed for async call if running directly

# Helper function to call the agent and print the final response
async def ask_agent(query: str):
    print(f"\n>>> User: {query}")
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not provide a final text response."
    agent_responder = "System" # Default if no agent responds clearly

    # Use run_async for potential async operations within agents/tools
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        # You can add more detailed event logging here if desired
        # print(f"  Event: Author={event.author}, Content={event.content}, Actions={event.actions}")
        if event.is_final_response() and event.content and event.content.parts:
             # Check if the text part exists before accessing
             if event.content.parts[0].text:
                  final_response_text = event.content.parts[0].text.strip()
                  agent_responder = event.author # Capture who gave the final response
                  break # Exit after getting the first final text response

    print(f"<<< {agent_responder}: {final_response_text}")


# --- Run Interactions ---

# Wrapper to run async function in different environments
async def run_interactions():
    # Example 1: Weather query (handled by weather_agent)
    await ask_agent("What's the weather like in Chennai?")

    # Example 2: Greeting (should be transferred to greeting_agent)
    await ask_agent("Hello there!")

    # Example 3: Weather query after greeting (should transfer back to weather_agent)
    await ask_agent("How about the weather in Toronto?")

    # Example 4: Farewell (should be transferred to farewell_agent)
    await ask_agent("Okay, thanks, bye!")

# Run the interactions
await run_interactions()


>>> User: What's the weather like in Chennai?


<<< weather_agent: The weather in Chennai is 15 degrees Celsius, rainy with a cloudy sky.

>>> User: Hello there!


<<< greeting_agent: Hello, there!

>>> User: How about the weather in Toronto?


<<< greeting_agent: Hello, there! I can greet you, but I cannot provide weather information. Would you like me to find an agent that can?

>>> User: Okay, thanks, bye!
<<< greeting_agent: Goodbye!


### LLM Agents with Callbacks (Agent, Model & Tool)

In [ ]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
import asyncio

# Constant
APP_NAME = "weather_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "weather_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"


# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."


def get_greeting(name: str) -> str:
    """Greets the given name.

    Args:
        name: The name to greet.

    Returns:
        A greeting message.
    """
    return f"Hello, {name}!"


# Callbacks

def before_model_callback(callback_context, llm_request):
    print(f"Before Model Callback: Agent {callback_context._invocation_context.agent.name}, Request: {llm_request.contents}")
    return None

def after_model_callback(callback_context, llm_response):
    print(f"After Model Callback: Agent {callback_context._invocation_context.agent.name}, Response: {llm_response.content}")
    return None

def before_tool_callback(tool, args, tool_context):
    print(f"Before Tool Callback: Tool {tool.name}, Args: {args}")
    return None

def after_tool_callback(tool, args, tool_context, tool_response):
    print(f"After Tool Callback: Tool {tool.name}, Response: {tool_response}")
    return None

def before_agent_callback(callback_context):
    print(f"Before Agent Callback: Agent {callback_context._invocation_context.agent.name}")
    return None

def after_agent_callback(callback_context):
    print(f"After Agent Callback: Agent {callback_context._invocation_context.agent.name}")
    return None


weather_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name="weather_agent",
    instruction="""You are a Weather Information Agent. Your task is to provide weather information for a given city.

    When a user provides a prompt, extract the city name.
    Then, use the `get_weather` tool to retrieve the weather information for that city.
    Finally, present the weather information to the user in a clear and concise manner.
    If the user asks for a greeting, transfer to the greeting agent.""",
    description="""You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task.""",
    tools=[get_weather],
    allow_transfer=True,
    before_model_callback=before_model_callback,
    after_model_callback=after_model_callback,
    before_tool_callback=before_tool_callback,
    after_tool_callback=after_tool_callback,
    before_agent_callback=before_agent_callback,
    after_agent_callback=after_agent_callback,
)

greeting_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name="greeting_agent",
    instruction="""You are a Greeting Agent. Your task is to greet the user.

    When a user provides a prompt, extract the name.
    Then, use the `get_greeting` tool to greet the user.
    Finally, present the greeting to the user in a clear and concise manner.
    If the user asks for weather information, transfer to the weather agent.""",
    description="""You are an agent who can greet a user.
    You have access to the `get_greeting` tool to accomplish this task.""",
    tools=[get_greeting],
    allow_transfer=True,
    before_model_callback=before_model_callback,
    after_model_callback=after_model_callback,
    before_tool_callback=before_tool_callback,
    after_tool_callback=after_tool_callback,
    before_agent_callback=before_agent_callback,
    after_agent_callback=after_agent_callback,
)

# Set parent-child relationship
weather_agent.children = [greeting_agent]
greeting_agent.parent_agent = weather_agent

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=weather_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
async def call_agent_async(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

In [ ]:
await call_agent_async("What's the weather in Chicago?")

Before Agent Callback: Agent weather_agent
Before Model Callback: Agent weather_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user')]


After Model Callback: Agent weather_agent, Response: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Chicago'}, name='get_weather'), function_response=None, inline_data=None, text=None)] role='model'
Before Tool Callback: Tool get_weather, Args: {'city': 'Chicago'}
After Tool Callback: Tool get_weather, Response: Weather in Chicago is 25 degrees Celsius, sunny with a clear sky.
Before Model Callback: Agent weather_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Chicago'}, name='get_weather'), functio

In [ ]:
await call_agent_async("Hello, there!")

Before Agent Callback: Agent weather_agent
Before Model Callback: Agent weather_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Chicago'}, name='get_weather'), function_response=None, inline_data=None, text=None)], role='model'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id=None, name='get_weather', response={'result': 'Weather in Chicago is 25 degrees Celsius, sunny with a clear sky.'}), inline_data=None, text=None)], role='user'), Content(parts=[Part(video_metadata=None, thought=N

After Model Callback: Agent weather_agent, Response: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'agent_name': 'greeting_agent'}, name='transfer_to_agent'), function_response=None, inline_data=None, text=None)] role='model'
Before Tool Callback: Tool transfer_to_agent, Args: {'agent_name': 'greeting_agent'}
After Tool Callback: Tool transfer_to_agent, Response: {}
Before Agent Callback: Agent greeting_agent
Before Model Callback: Agent greeting_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, te

After Model Callback: Agent greeting_agent, Response: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'name': 'there'}, name='get_greeting'), function_response=None, inline_data=None, text=None)] role='model'
Before Tool Callback: Tool get_greeting, Args: {'name': 'there'}
After Tool Callback: Tool get_greeting, Response: Hello, there!
Before Model Callback: Agent greeting_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='OK. The weather in Chicago is 25 degrees Celsius, sunny with a clear sky.

In [ ]:
await call_agent_async("How about Toronto?")

Before Agent Callback: Agent greeting_agent
Before Model Callback: Agent greeting_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='OK. The weather in Chicago is 25 degrees Celsius, sunny with a clear sky.\n')], role='model'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Hello, there!')], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, a

After Model Callback: Agent greeting_agent, Response: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'agent_name': 'weather_agent'}, name='transfer_to_agent'), function_response=None, inline_data=None, text=None)] role='model'
Before Tool Callback: Tool transfer_to_agent, Args: {'agent_name': 'weather_agent'}
After Tool Callback: Tool transfer_to_agent, Response: {}
Before Agent Callback: Agent weather_agent
Before Model Callback: Agent weather_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Chicago'}, name=

After Model Callback: Agent weather_agent, Response: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Toronto'}, name='get_weather'), function_response=None, inline_data=None, text=None)] role='model'
Before Tool Callback: Tool get_weather, Args: {'city': 'Toronto'}
After Tool Callback: Tool get_weather, Response: Weather in Toronto is 30 degrees Celsius, partly cloudy with a overcast sky.
Before Model Callback: Agent weather_agent, Request: [Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="What's the weather in Chicago?")], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'city': 'Chicago'}, name='get_weather

### LLM Agent with before_agent_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional
from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define the Callback Function (Same as before) ---
def simple_before_agent_logger(callback_context: CallbackContext) -> Optional[types.Content]:
    """Logs entry into an agent and checks a condition."""
    agent_name = callback_context.agent_name
    invocation_id = callback_context.invocation_id
    print(f"[Callback] Entering agent: {agent_name} (Invocation: {invocation_id})")

    # Example: Check a condition in state
    if callback_context.state.get("skip_agent", False):
        print(f"[Callback] Condition met: Skipping agent {agent_name}.")
        # Return Content to skip the agent's run
        return types.Content(parts=[types.Part(text=f"Agent {agent_name} was skipped by callback.")])
    else:
        print(f"[Callback] Condition not met: Proceeding with agent {agent_name}.")
        # Return None to allow the agent's run to execute
        return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent and Assign Callback
    my_llm_agent = LlmAgent(
        name="SimpleLlmAgent",
        model=GEMINI_2_FLASH,
        instruction="You are a simple agent. Just say 'Hello!'",
        description="An LLM agent demonstrating before_agent_callback",
        before_agent_callback=simple_before_agent_logger
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_demo_app", session_service=session_service)
    session_id_run = "llm_session_run_1"
    session_id_skip = "llm_session_skip_1"
    user_id = "llm_test_user"

    # Create sessions
    session_service.create_session(app_name="llm_demo_app", user_id=user_id, session_id=session_id_run)
    session_service.create_session(app_name="llm_demo_app", user_id=user_id, session_id=session_id_skip,
                                  state={"skip_agent": True}) # Set state to trigger skip condition

    print("--- Running LLM Agent Normally ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Run normally")])):
        # Only print final LLM response or callback override
        if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}") # Added strip() for cleaner output

    print("\n--- Running LLM Agent with Skip Condition ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_skip, new_message=types.Content(parts=[types.Part(text="Skip this agent")])):
         # Only print final LLM response or callback override
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")



await main()

--- Running LLM Agent Normally ---
[Callback] Entering agent: SimpleLlmAgent (Invocation: e-b4399c56-6c14-40b6-b339-ff2100bdcff6)
[Callback] Condition not met: Proceeding with agent SimpleLlmAgent.
Event Output: SimpleLlmAgent: Hello!

--- Running LLM Agent with Skip Condition ---
[Callback] Entering agent: SimpleLlmAgent (Invocation: e-fe4f9989-3376-4707-88a8-2b135dc01a89)
[Callback] Condition met: Skipping agent SimpleLlmAgent.
Event Output: SimpleLlmAgent: Agent SimpleLlmAgent was skipped by callback.


### LLM Agent with after_agent_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional
from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define the Callback Function ---
def simple_after_agent_logger(callback_context: CallbackContext) -> Optional[types.Content]:
    """Logs exit from an agent and optionally appends a message."""
    agent_name = callback_context.agent_name
    invocation_id = callback_context.invocation_id
    print(f"[Callback] Exiting agent: {agent_name} (Invocation: {invocation_id})")

    # Example: Check state potentially modified during the agent's run
    final_status = callback_context.state.get("agent_run_status", "Completed Normally")
    print(f"[Callback] Agent run status from state: {final_status}")

    # Example: Optionally return Content to append a message
    if callback_context.state.get("add_concluding_note", False):
        print(f"[Callback] Adding concluding note for agent {agent_name}.")
        # Return Content to append after the agent's own output
        return types.Content(parts=[types.Part(text=f"Concluding note added by after_agent_callback.")])
    else:
        print(f"[Callback] No concluding note added for agent {agent_name}.")
        # Return None - no additional message appended
        return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent and Assign Callback
    my_llm_agent = LlmAgent(
        name="SimpleLlmAgentWithAfter",
        model=GEMINI_2_FLASH,
        instruction="You are a simple agent. Just say 'Processing complete!'",
        description="An LLM agent demonstrating after_agent_callback",
        after_agent_callback=simple_after_agent_logger # Assign the function here
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_demo_app_after", session_service=session_service)
    session_id_run = "llm_session_run_after_1"
    session_id_conclude = "llm_session_conclude_1"
    user_id = "llm_test_user_after"

    # Create sessions
    session_service.create_session(app_name="llm_demo_app_after", user_id=user_id, session_id=session_id_run)
    # Session where the callback will add a note
    session_service.create_session(app_name="llm_demo_app_after", user_id=user_id, session_id=session_id_conclude,
                                  state={"add_concluding_note": True})

    print("--- Running LLM Agent Normally (with after_agent_callback) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Run normally")])):
        # Print any event content from agent or callback
        if event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running LLM Agent with Concluding Note Condition ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_conclude,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Run and conclude")])):
        # Print any event content from agent or callback
         if event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")


await main()

--- Running LLM Agent Normally (with after_agent_callback) ---
Event Output: SimpleLlmAgentWithAfter: Processing complete!
[Callback] Exiting agent: SimpleLlmAgentWithAfter (Invocation: e-f5a5021a-eb55-41d4-9273-ab41630292e5)
[Callback] Agent run status from state: Completed Normally
[Callback] No concluding note added for agent SimpleLlmAgentWithAfter.

--- Running LLM Agent with Concluding Note Condition ---
Event Output: SimpleLlmAgentWithAfter: Processing complete!
[Callback] Exiting agent: SimpleLlmAgentWithAfter (Invocation: e-98aac661-9c0d-463c-ac4d-c31ef4d3d419)
[Callback] Agent run status from state: Completed Normally
[Callback] Adding concluding note for agent SimpleLlmAgentWithAfter.
Event Output: SimpleLlmAgentWithAfter: Concluding note added by after_agent_callback.


### LLM Agent with before_model_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional
from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
# Need LlmRequest and LlmResponse for the callback signature and return type
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define the Callback Function ---
def simple_before_model_modifier(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Inspects/modifies the LLM request or skips the call."""
    agent_name = callback_context.agent_name
    print(f"[Callback] Before model call for agent: {agent_name}")

    # Inspect the last user message in the request contents
    last_user_message = ""
    if llm_request.contents and llm_request.contents[-1].role == 'user':
         if llm_request.contents[-1].parts:
            last_user_message = llm_request.contents[-1].parts[0].text
    print(f"[Callback] Inspecting last user message: '{last_user_message}'")

    # --- Modification Example ---
    # Add a prefix to the system instruction
    original_instruction = llm_request.config.system_instruction or types.Content(role="system", parts=[])
    prefix = "[Modified by Callback] "
    # Ensure system_instruction is Content and parts list exists
    if not isinstance(original_instruction, types.Content):
         # Handle case where it might be a string (though config expects Content)
         original_instruction = types.Content(role="system", parts=[types.Part(text=str(original_instruction))])
    if not original_instruction.parts:
        original_instruction.parts.append(types.Part(text="")) # Add an empty part if none exist

    # Modify the text of the first part
    modified_text = prefix + (original_instruction.parts[0].text or "")
    original_instruction.parts[0].text = modified_text
    llm_request.config.system_instruction = original_instruction
    print(f"[Callback] Modified system instruction to: '{modified_text}'")


    # --- Skip Example ---
    # Check if the last user message contains "BLOCK"
    if "BLOCK" in last_user_message.upper():
        print("[Callback] 'BLOCK' keyword found. Skipping LLM call.")
        # Return an LlmResponse to skip the actual LLM call
        return LlmResponse(
            content=types.Content(
                role="model",
                parts=[types.Part(text="LLM call was blocked by before_model_callback.")],
            )
        )
    else:
        print("[Callback] Proceeding with LLM call.")
        # Return None to allow the (modified) request to go to the LLM
        return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent and Assign Callback
    my_llm_agent = LlmAgent(
        name="ModelCallbackAgent",
        model=GEMINI_2_FLASH,
        instruction="You are a helpful assistant.", # Base instruction
        description="An LLM agent demonstrating before_model_callback",
        before_model_callback=simple_before_model_modifier # Assign the function here
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_model_cb_app", session_service=session_service)
    session_id_run = "model_cb_run_1"
    session_id_block = "model_cb_block_1"
    user_id = "model_cb_user"

    # Create sessions
    session_service.create_session(app_name="llm_model_cb_app", user_id=user_id, session_id=session_id_run)
    session_service.create_session(app_name="llm_model_cb_app", user_id=user_id, session_id=session_id_block)

    print("--- Running LLM Agent Normally (with before_model_callback modification) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Tell me a short joke.")])):
        # Only print final LLM response or callback override
        if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running LLM Agent with BLOCK Keyword (triggering skip) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_block,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="BLOCK this request.")])):
         # Only print final LLM response or callback override
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")


await main()

--- Running LLM Agent Normally (with before_model_callback modification) ---
[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'Tell me a short joke.'
[Callback] Modified system instruction to: '[Modified by Callback] You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_callback"'
[Callback] Proceeding with LLM call.
Event Output: ModelCallbackAgent: Why don't scientists trust atoms?

Because they make up everything!

--- Running LLM Agent with BLOCK Keyword (triggering skip) ---
[Callback] Before model call for agent: ModelCallbackAgent
[Callback] Inspecting last user message: 'BLOCK this request.'
[Callback] Modified system instruction to: '[Modified by Callback] You are a helpful assistant.

You are an agent. Your internal name is "ModelCallbackAgent".

 The description about you is "An LLM agent demonstrating before_model_c

### LLM Agent with after_model_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional
import copy # Needed to safely modify response content

from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define the Callback Function ---
def simple_after_model_modifier(
    callback_context: CallbackContext, llm_response: LlmResponse
) -> Optional[LlmResponse]:
    """Inspects/modifies the LLM response after it's received."""
    agent_name = callback_context.agent_name
    print(f"[Callback] After model call for agent: {agent_name}")

    # --- Inspection ---
    original_text = ""
    if llm_response.content and llm_response.content.parts:
        # Assuming simple text response for this example
        if llm_response.content.parts[0].text:
            original_text = llm_response.content.parts[0].text
            print(f"[Callback] Inspected original response text: '{original_text[:100]}...'") # Log snippet
        elif llm_response.content.parts[0].function_call:
             print(f"[Callback] Inspected response: Contains function call '{llm_response.content.parts[0].function_call.name}'. No text modification.")
             return None # Don't modify tool calls in this example
        else:
             print("[Callback] Inspected response: No text content found.")
             return None
    elif llm_response.error_message:
        print(f"[Callback] Inspected response: Contains error '{llm_response.error_message}'. No modification.")
        return None
    else:
        print("[Callback] Inspected response: Empty LlmResponse.")
        return None # Nothing to modify

    # --- Modification Example ---
    # Replace "scientists" with "funny scientists" (case-insensitive)
    search_term = "scientists"
    replace_term = "funny scientists"
    if search_term in original_text.lower():
        print(f"[Callback] Found '{search_term}'. Modifying response.")
        modified_text = original_text.replace(search_term, replace_term)
        modified_text = modified_text.replace(search_term.capitalize(), replace_term.capitalize()) # Handle capitalization

        # Create a NEW LlmResponse with the modified content
        # Deep copy parts to avoid modifying original if other callbacks exist
        modified_parts = [copy.deepcopy(part) for part in llm_response.content.parts]
        modified_parts[0].text = modified_text # Update the text in the copied part

        new_response = LlmResponse(
             content=types.Content(role="model", parts=modified_parts),
             # Copy other relevant fields if necessary, e.g., grounding_metadata
             grounding_metadata=llm_response.grounding_metadata
             )
        print(f"[Callback] Returning modified response.")
        return new_response # Return the modified response
    else:
        print(f"[Callback] '{search_term}' not found. Passing original response through.")
        # Return None to use the original llm_response
        return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent and Assign Callback
    my_llm_agent = LlmAgent(
        name="AfterModelCallbackAgent",
        model=GEMINI_2_FLASH,
        instruction="You are a helpful assistant.",
        description="An LLM agent demonstrating after_model_callback",
        after_model_callback=simple_after_model_modifier # Assign the function here
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_after_model_cb_app", session_service=session_service)
    session_id_run = "after_model_cb_run_1"
    session_id_modify = "after_model_cb_modify_1"
    user_id = "after_model_cb_user"

    # Create sessions
    session_service.create_session(app_name="llm_after_model_cb_app", user_id=user_id, session_id=session_id_run)
    session_service.create_session(app_name="llm_after_model_cb_app", user_id=user_id, session_id=session_id_modify)

    print("--- Running LLM Agent Normally (Callback passes response through) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Say hello.")])):
        # Only print final LLM response
        if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running LLM Agent with Input Triggering Modification ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_modify,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="Tell me a short joke.")])):
         # Only print final LLM response
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

await main()

--- Running LLM Agent Normally (Callback passes response through) ---
[Callback] After model call for agent: AfterModelCallbackAgent
[Callback] Inspected original response text: 'Hello! I'm AfterModelCallbackAgent, an LLM agent designed to demonstrate the use of `after_model_cal...'
[Callback] 'scientists' not found. Passing original response through.
Event Output: AfterModelCallbackAgent: Hello! I'm AfterModelCallbackAgent, an LLM agent designed to demonstrate the use of `after_model_callback`. I'm here to help you explore how to use this feature!

--- Running LLM Agent with Input Triggering Modification ---
[Callback] After model call for agent: AfterModelCallbackAgent
[Callback] Inspected original response text: 'Why don't scientists trust atoms? 

Because they make up everything!
...'
[Callback] Found 'scientists'. Modifying response.
[Callback] Returning modified response.
Event Output: AfterModelCallbackAgent: Why don't funny scientists trust atoms? 

Because they make up everyth

### LLM Agent with before_tool_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional, Dict, Any
import copy

from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
# Need LlmRequest, LlmResponse for context
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
# Need BaseTool, ToolContext for the callback signature
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
# Using FunctionTool to easily create a tool
from google.adk.tools.function_tool import FunctionTool
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define a Simple Tool Function ---
def get_capital_city(country: str) -> str:
    """Retrieves the capital city of a given country."""
    print(f"--- Tool 'get_capital_city' executing with country: {country} ---")
    country_capitals = {
        "united states": "Washington, D.C.",
        "canada": "Ottawa", # Intentionally correct here
        "france": "Paris",
        "germany": "Berlin",
    }
    return country_capitals.get(country.lower(), f"Capital not found for {country}")

# --- Wrap the function into a Tool ---
capital_tool = FunctionTool(func=get_capital_city)

# --- Define the Callback Function ---
def simple_before_tool_modifier(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """Inspects/modifies tool args or skips the tool call."""
    agent_name = tool_context.agent_name
    tool_name = tool.name
    print(f"[Callback] Before tool call for tool '{tool_name}' in agent '{agent_name}'")
    print(f"[Callback] Original args: {args}")

    # --- Modification Example ---
    # If the tool is 'get_capital_city' and country is 'Canada', change it to 'France'
    if tool_name == 'get_capital_city' and args.get('country', '').lower() == 'canada':
        print("[Callback] Detected 'Canada'. Modifying args to 'France'.")
        args['country'] = 'France' # Modify the args dictionary directly
        print(f"[Callback] Modified args: {args}")
        return None # Proceed with modified args

    # --- Skip Example ---
    # If the tool is 'get_capital_city' and country is 'BLOCK'
    if tool_name == 'get_capital_city' and args.get('country', '').upper() == 'BLOCK':
        print("[Callback] Detected 'BLOCK'. Skipping tool execution.")
        # Return a dictionary to be used as the tool result, skipping the actual tool call
        return {"result": "Tool execution was blocked by before_tool_callback."}

    print("[Callback] Proceeding with original or previously modified args.")
    # Return None to allow the tool to execute normally (with original or modified args)
    return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent with the tool and callback
    my_llm_agent = LlmAgent(
        name="ToolCallbackAgent",
        model=GEMINI_2_FLASH,
        instruction="You are an agent that can find capital cities. Use the get_capital_city tool.",
        description="An LLM agent demonstrating before_tool_callback",
        tools=[capital_tool], # Add the tool here
        before_tool_callback=simple_before_tool_modifier # Assign the callback here
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_tool_cb_app", session_service=session_service)
    session_id_run = "tool_cb_run_1"
    session_id_modify = "tool_cb_modify_1"
    session_id_block = "tool_cb_block_1"
    user_id = "tool_cb_user"

    # Create sessions
    session_service.create_session(app_name="llm_tool_cb_app", user_id=user_id, session_id=session_id_run)
    session_service.create_session(app_name="llm_tool_cb_app", user_id=user_id, session_id=session_id_modify)
    session_service.create_session(app_name="llm_tool_cb_app", user_id=user_id, session_id=session_id_block)

    print("--- Running Agent (Normal Tool Call - Germany) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="What is the capital of Germany?")])):
        # Only print final LLM response
        if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running Agent (Tool Call Triggering Modification - Canada -> France) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_modify,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="What is the capital of Canada?")])):
         # Only print final LLM response
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running Agent (Tool Call Triggering Skip - BLOCK) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_block,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="What is the capital of BLOCK?")])):
         # Only print final LLM response
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

await main()

--- Running Agent (Normal Tool Call - Germany) ---


[Callback] Before tool call for tool 'get_capital_city' in agent 'ToolCallbackAgent'
[Callback] Original args: {'country': 'Germany'}
[Callback] Proceeding with original or previously modified args.
--- Tool 'get_capital_city' executing with country: Germany ---
Event Output: ToolCallbackAgent: The capital of Germany is Berlin.

--- Running Agent (Tool Call Triggering Modification - Canada -> France) ---


[Callback] Before tool call for tool 'get_capital_city' in agent 'ToolCallbackAgent'
[Callback] Original args: {'country': 'Canada'}
[Callback] Detected 'Canada'. Modifying args to 'France'.
[Callback] Modified args: {'country': 'France'}
--- Tool 'get_capital_city' executing with country: France ---
Event Output: ToolCallbackAgent: The capital of France is Paris.

--- Running Agent (Tool Call Triggering Skip - BLOCK) ---
Event Output: ToolCallbackAgent: I cannot fulfill that request. The tool requires a valid country name, and 'BLOCK' is not a recognized country. Can you provide me with a valid country name?


### LLM Agent with after_tool_callback and state

In [ ]:
import asyncio
from typing import AsyncGenerator, Optional, Dict, Any
import copy # Good practice for modifying results

from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.function_tool import FunctionTool
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event
from google.genai import types

# Ensure GEMINI_2_FLASH is defined (replace if needed)
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred model

# --- Define a Simple Tool Function (Same as before) ---
def get_capital_city(country: str) -> str:
    """Retrieves the capital city of a given country."""
    print(f"--- Tool 'get_capital_city' executing with country: {country} ---")
    country_capitals = {
        "united states": "Washington, D.C.",
        "canada": "Ottawa",
        "france": "Paris",
        "germany": "Berlin",
    }
    return {"result": country_capitals.get(country.lower(), f"Capital not found for {country}")}

# --- Wrap the function into a Tool ---
capital_tool = FunctionTool(func=get_capital_city)

# --- Define the Callback Function ---
def simple_after_tool_modifier(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext, tool_response: Dict
) -> Optional[Dict]:
    """Inspects/modifies the tool result after execution."""
    agent_name = tool_context.agent_name
    tool_name = tool.name
    print(f"[Callback] After tool call for tool '{tool_name}' in agent '{agent_name}'")
    print(f"[Callback] Args used: {args}")
    print(f"[Callback] Original tool_response: {tool_response}")

    # Default structure for function tool results is {"result": <return_value>}
    original_result_value = tool_response.get("result", "")
    # original_result_value = tool_response

    # --- Modification Example ---
    # If the tool was 'get_capital_city' and result is 'Washington, D.C.'
    if tool_name == 'get_capital_city' and original_result_value == "Washington, D.C.":
        print("[Callback] Detected 'Washington, D.C.'. Modifying tool response.")

        # IMPORTANT: Create a new dictionary or modify a copy
        modified_response = copy.deepcopy(tool_response)
        modified_response["result"] = f"{original_result_value} (Note: This is the capital of the USA)."
        modified_response["note_added_by_callback"] = True # Add extra info if needed

        print(f"[Callback] Modified tool_response: {modified_response}")
        return modified_response # Return the modified dictionary

    print("[Callback] Passing original tool response through.")
    # Return None to use the original tool_response
    return None

# --- Setup and Run ---
async def main():
    # 1. Create LlmAgent with the tool and callback
    my_llm_agent = LlmAgent(
        name="AfterToolCallbackAgent",
        model=GEMINI_2_FLASH,
        instruction="You are an agent that finds capital cities using the get_capital_city tool. Report the result clearly.",
        description="An LLM agent demonstrating after_tool_callback",
        tools=[capital_tool], # Add the tool
        after_tool_callback=simple_after_tool_modifier # Assign the callback
    )

    # 2. Setup Runner and Session
    session_service = InMemorySessionService()
    runner = Runner(agent=my_llm_agent, app_name="llm_after_tool_cb_app", session_service=session_service)
    session_id_run = "after_tool_cb_run_1"
    session_id_modify = "after_tool_cb_modify_1"
    user_id = "after_tool_cb_user"

    # Create sessions
    session_service.create_session(app_name="llm_after_tool_cb_app", user_id=user_id, session_id=session_id_run)
    session_service.create_session(app_name="llm_after_tool_cb_app", user_id=user_id, session_id=session_id_modify)

    print("--- Running Agent (Callback passes result through - France) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_run,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="What is the capital of France?")])):
        # Only print final LLM response
        if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

    print("\n--- Running Agent (Callback modifies result - United States) ---")
    async for event in runner.run_async(user_id=user_id, session_id=session_id_modify,
                                        new_message=types.Content(role="user",
                                                                  parts=[types.Part(text="What is the capital of the United States?")])):
         # Only print final LLM response
         if event.is_final_response() and event.content:
            print(f"Event Output: {event.author}: {event.content.parts[0].text.strip()}")

await main()

--- Running Agent (Callback passes result through - France) ---


--- Tool 'get_capital_city' executing with country: France ---
[Callback] After tool call for tool 'get_capital_city' in agent 'AfterToolCallbackAgent'
[Callback] Args used: {'country': 'France'}
[Callback] Original tool_response: {'result': 'Paris'}
[Callback] Passing original tool response through.
Event Output: AfterToolCallbackAgent: The capital of France is Paris.

--- Running Agent (Callback modifies result - United States) ---


--- Tool 'get_capital_city' executing with country: United States ---
[Callback] After tool call for tool 'get_capital_city' in agent 'AfterToolCallbackAgent'
[Callback] Args used: {'country': 'United States'}
[Callback] Original tool_response: {'result': 'Washington, D.C.'}
[Callback] Detected 'Washington, D.C.'. Modifying tool response.
[Callback] Modified tool_response: {'result': 'Washington, D.C. (Note: This is the capital of the USA).', 'note_added_by_callback': True}
Event Output: AfterToolCallbackAgent: The capital of the United States is Washington, D.C. (Note: This is the capital of the USA).


### LLM Agent with Gaurdrail (Profanity Checker with before_model callback)

In [ ]:
from google.adk.agents.callback_context import CallbackContext
from google.adk.agents.llm_agent import AfterModelCallback, BeforeModelCallback
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.agents.llm_agent import LlmAgent
from typing import Any, List, Optional
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent


def profanity_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Check for profanity in the model request."""
    profanity_list: List[str] = ["badword1", "badword2", "badword3"]
    if llm_request.contents:
        for content in llm_request.contents:
            for part in content.parts:
                if part.text:
                    for profanity in profanity_list:
                        if profanity in part.text.lower():
                            callback_context.state["profanity_trigger"] = True
                            return LlmResponse(
                                content=types.Content(
                                    role="model",
                                    parts=[
                                        types.Part(
                                            text=(
                                                "No bad word allowed."
                                            )
                                        )
                                    ],
                                )
                            )
    return None


# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."

async def run_query(query: str):
    weather_agent = LlmAgent(
        model=GEMINI_2_FLASH,
        name="weather_agent",
        instruction="""You are a Weather Information Agent. Your task is to provide weather information for a given city.

        When a user provides a prompt, extract the city name.
        Then, use the `get_weather` tool to retrieve the weather information for that city.
        Finally, present the weather information to the user in a clear and concise manner.
        If the user asks for a greeting, transfer to the greeting agent.""",
        description="""You are an agent who can fetch weather information for a city.
        You have access to the `get_weather` tool to accomplish this task.""",
        tools=[get_weather],
        before_model_callback=profanity_guardrail,
    )

    session_service = InMemorySessionService()
    session = session_service.create_session(
        app_name="weather_app", user_id="12345", session_id="123344"
    )
    runner = Runner(
        agent=weather_agent,
        app_name="weather_app",
        session_service=session_service,
    )

    content = types.Content(role="user", parts=[types.Part(text=query)])
    async for event in runner.run_async(
        user_id="12345", session_id="123344", new_message=content
    ):
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print(f"Query: {query}")
            print(f"Agent Response: {final_response}")
            print("-" * 20)
        if "profanity_trigger" in event.actions.state_delta:
            print(f"Profanity Triggered: {event.actions.state_delta['profanity_trigger']}")


In [ ]:
await run_query("What is the weather in Chicago?")

Query: What is the weather in Chicago?
Agent Response: The weather in Chicago is 25 degrees Celsius, sunny with a clear sky.

--------------------


In [ ]:
await run_query("what the badword1 is the weather in Chicago?")

Query: what the badword1 is the weather in Chicago?
Agent Response: No bad word allowed.
--------------------
Profanity Triggered: True


### LlmAgent with All Callbacks

In [ ]:
import asyncio
import warnings
import os
from typing import Any, Optional, Dict, List, AsyncGenerator

# --- ADK Imports ---
from google.adk.agents import Agent, LlmAgent, BaseAgent # Using LlmAgent directly
from google.adk.sessions import InMemorySessionService, Session, State
from google.adk.runners import Runner
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.function_tool import FunctionTool
from google.adk.events import Event
from google.adk.agents.invocation_context import InvocationContext
from google.adk.agents.callback_context import CallbackContext
from google.genai import types

# Suppress specific UserWarning from google.generativeai if necessary
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types')

# --- Constants ---
APP_NAME = "support_ticket_app"
USER_ID = "customer_123"
SESSION_ID = "ticket_session_abc"
AGENT_NAME = "support_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or your preferred Gemini model

# Ensure Project and Location are set if using Vertex AI backend
# Example:
# os.environ["GOOGLE_CLOUD_PROJECT"] = "your-gcp-project-id"
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"


# --- Simulated Knowledge Base Tool ---
def kb_search(keywords: List[str]) -> Dict[str, Any]:
    """
    Searches the knowledge base for troubleshooting steps based on keywords.

    Args:
        keywords: A list of keywords related to the issue (e.g., ['screen', 'flickering']).

    Returns:
        A dictionary containing potential solutions or an empty dictionary if none found.
    """
    print(f"      [Tool Executing: kb_search with keywords: {keywords}]")
    # Simple mock implementation
    mock_kb = {
        "screen": ["Check the display cable connection.", "Try a different monitor port."],
        "flickering": ["Update the graphics driver.", "Adjust the screen refresh rate."],
        "display": ["Ensure monitor power is on.", "Reboot the computer."],
        "keyboard": ["Check battery if wireless.", "Try a different USB port."]
    }
    results = []
    for keyword in keywords:
        if keyword.lower() in mock_kb:
            results.extend(mock_kb[keyword.lower()])

    if results:
        # Remove duplicates while preserving order (if Python 3.7+)
        unique_results = list(dict.fromkeys(results))
        print(f"      [Tool Result: Found {len(unique_results)} steps]")
        return {"solutions": unique_results}
    else:
        print("      [Tool Result: No relevant KB articles found]")
        return {"solutions": ["No specific troubleshooting steps found in KB for these keywords."]}

# Wrap the function into a FunctionTool
kb_search_tool = FunctionTool(func=kb_search)


# --- Callback Implementations ---

def log_before_agent(callback_context: CallbackContext) -> Optional[types.Content]:
    """Callback executed before the agent starts processing."""
    ticket_id = f"TICKET-{SESSION_ID.split('_')[-1].upper()}" # Simulate getting ticket ID
    print(f"\n[Callback Triggered: before_agent_callback]")
    print(f"  -> Processing Ticket ID: {ticket_id}")
    # Example state modification: Store ticket ID
    callback_context.state["ticket_id"] = ticket_id
    print(f"  -> Agent '{callback_context.agent_name}' starting.")
    return None # Return None to allow agent execution

def log_after_agent(callback_context: CallbackContext) -> Optional[types.Content]:
    """Callback executed after the agent finishes processing."""
    ticket_id = callback_context.state.get("ticket_id", "UNKNOWN")
    print(f"\n[Callback Triggered: after_agent_callback]")
    print(f"  -> Finished processing for Ticket ID: {ticket_id}.")
    # Example: Simulate updating ticket status in an external system
    print(f"  -> Updating external system: Ticket {ticket_id} status set to 'Responded'.")
    # Example state modification
    callback_context.state["processing_status"] = "completed"
    return None # Return None, we don't want to append extra content here

def log_before_model(callback_context: CallbackContext, llm_request: LlmRequest) -> Optional[LlmResponse]:
    """Callback executed before sending the request to the LLM."""
    print(f"\n[Callback Triggered: before_model_callback]")
    print(f"  -> Preparing to call LLM for agent '{callback_context.agent_name}'.")
    # Example: Log request details (be careful with PII in real scenarios)
    # print(f"  -> LLM Request Contents (brief): {str(llm_request.contents)[:200]}...")
    # Example: Add a safety reminder (Note: modifying system_instruction directly might be overwritten)
    # llm_request.append_instructions(["Remember to be helpful and safe."]) # Use append_instructions
    print(f"  -> Safety check/prompt augmentation applied (simulated).")
    return None # Return None to proceed with LLM call

def check_after_model(callback_context: CallbackContext, llm_response: LlmResponse) -> Optional[LlmResponse]:
    """Callback executed after receiving the response from the LLM."""
    print(f"\n[Callback Triggered: after_model_callback]")
    print(f"  -> Received LLM response for agent '{callback_context.agent_name}'.")
    # Example: Log response details (be careful with PII)
    response_text = llm_response.content.parts[0].text if llm_response.content and llm_response.content.parts else "[No Text]"
    print(f"  -> LLM Raw Response (brief): {response_text[:100]}...")
    # Example: Simulate PII check
    if "password" in response_text.lower() or "credit card" in response_text.lower():
        print("  -> !! PII potentially detected in LLM response (simulated) !!")
        # Could modify response here, e.g., return an error or redacted text
        # return LlmResponse(content=types.Content(parts=[types.Part(text="[Response redacted due to potential PII]")]))
    else:
        print("  -> PII check passed (simulated).")
    return None # Return None to use the original (or potentially modified) LLM response

def validate_before_tool(tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext) -> Optional[Dict]:
    """Callback executed before a tool is called."""
    print(f"\n[Callback Triggered: before_tool_callback for Tool: '{tool.name}']")
    print(f"  -> Attempting to call tool '{tool.name}' with args: {args}")
    # Example: Validate arguments
    if tool.name == "kb_search" and "keywords" in args:
        if not isinstance(args["keywords"], list) or not args["keywords"]:
            print("  -> !! Validation Failed: Keywords must be a non-empty list. Skipping tool call. !!")
            return {"error": "Invalid keywords provided for KB search."} # Return error to LLM
        print("  -> Keyword validation passed.")
    return None # Return None to proceed with actual tool execution

def log_after_tool(tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext, tool_response: Dict) -> Optional[Dict]:
    """Callback executed after a tool has run."""
    print(f"\n[Callback Triggered: after_tool_callback for Tool: '{tool.name}']")
    print(f"  -> Tool '{tool.name}' executed with args: {args}")
    print(f"  -> Received tool response (brief): {str(tool_response)[:200]}...")
    # Example: Caching simulation (just log it)
    print(f"  -> Caching tool result (simulated).")
    # Example: Modify response if needed
    # if "solutions" in tool_response and tool_response["solutions"]:
    #    tool_response["solutions"].append("Also, try restarting your device.") # Append suggestion
    return None # Return None to use the original (or modified) tool response


# --- Agent Definition ---
support_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are an IT Support Agent. Your goal is to help users troubleshoot technical issues.
1. Analyze the user's problem description (support ticket).
2. Identify keywords related to the issue.
3. If the issue relates to common hardware problems like 'screen', 'display', 'flickering', 'keyboard', use the `kb_search` tool with the identified keywords to find troubleshooting steps.
4. Based on your analysis and any results from the `kb_search` tool, provide a clear, step-by-step response to the user.
5. If the `kb_search` tool doesn't return useful information, state that and provide general troubleshooting advice (e.g., restart, check connections).
""",
    description="First-level IT support agent that analyzes issues and uses a knowledge base.",
    tools=[kb_search_tool],
    allow_transfer=False,

    # --- Assign Callbacks ---
    before_agent_callback=log_before_agent,
    after_agent_callback=log_after_agent,
    before_model_callback=log_before_model,
    after_model_callback=check_after_model,
    before_tool_callback=validate_before_tool,
    after_tool_callback=log_after_tool,
)

# --- Session and Runner Setup ---
session_service = InMemorySessionService()
# Ensure session is created before running
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=support_agent, app_name=APP_NAME, session_service=session_service)

# --- Agent Interaction Logic ---
async def call_agent_and_show_flow(query):
  print(f"\n--- Starting Workflow for Query: '{query}' ---")
  print(f"\n[User Submits Ticket: '{query}']")
  content = types.Content(role='user', parts=[types.Part(text=query)])
  final_response_text = "[Agent did not produce a final response text]"

  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      # Print event details to trace the flow
      # print(f"\nDEBUG Event: Author={event.author}, Partial={event.partial}, Final={event.is_final_response()}, Content={str(event.content)[:100]}...")

      if event.get_function_calls():
           print(f"\n[AFW: LLM decided to use Tool '{event.get_function_calls()[0].name}']")
           # Before tool callback is triggered internally by the runner/flow

      elif event.get_function_responses():
           print(f"\n[AFW: Received result from Tool '{event.get_function_responses()[0].name}']")
           # After tool callback is triggered internally by the runner/flow
           print("  -> AFW: Sending tool result back to LLM for final response generation...")

      if event.is_final_response() and event.content and event.content.parts:
          # Check if there's text before accessing parts[0]
          if event.content.parts[0].text:
               final_response_text = event.content.parts[0].text
               print(f"\n[AFW: Sending Final Response to User]")
               print(f"  -> Response: {final_response_text}")
          else:
               print(f"\n[AFW: Final Response - Non-text content received: {event.content.parts}]")
               final_response_text = "[Non-text final response]"
          # After agent callback will be triggered after this loop finishes internally

  print(f"\n--- Workflow Finished for Query: '{query}' ---")
  # Retrieve final state to show callback modification
  final_session = session_service.get_session(APP_NAME, USER_ID, SESSION_ID)
  print(f"Final Session State (example): ticket_id='{final_session.state.get('ticket_id')}', status='{final_session.state.get('processing_status')}'")
  return final_response_text


await call_agent_and_show_flow("Help! My computer screen keeps flickering constantly.")


--- Starting Workflow for Query: 'Help! My computer screen keeps flickering constantly.' ---

[User Submits Ticket: 'Help! My computer screen keeps flickering constantly.']

[Callback Triggered: before_agent_callback]
  -> Processing Ticket ID: TICKET-ABC
  -> Agent 'support_agent' starting.

[Callback Triggered: before_model_callback]
  -> Preparing to call LLM for agent 'support_agent'.
  -> Safety check/prompt augmentation applied (simulated).



[Callback Triggered: after_model_callback]
  -> Received LLM response for agent 'support_agent'.
  -> LLM Raw Response (brief): Okay, I understand your computer screen is flickering. I can help you with that.

...
  -> PII check passed (simulated).

[AFW: LLM decided to use Tool 'kb_search']

[Callback Triggered: before_tool_callback for Tool: 'kb_search']
  -> Attempting to call tool 'kb_search' with args: {'keywords': ['screen', 'flickering']}
  -> Keyword validation passed.
      [Tool Executing: kb_search with keywords: ['screen', 'flickering']]
      [Tool Result: Found 4 steps]

[Callback Triggered: after_tool_callback for Tool: 'kb_search']
  -> Tool 'kb_search' executed with args: {'keywords': ['screen', 'flickering']}
  -> Received tool response (brief): {'solutions': ['Check the display cable connection.', 'Try a different monitor port.', 'Update the graphics driver.', 'Adjust the screen refresh rate.']}...
  -> Caching tool result (simulated).

[AFW: Received result from To

"Okay, I have some troubleshooting steps for you. Please try the following:\n\n1.  **Check the display cable connection:** Make sure the cable connecting your monitor to your computer is securely plugged in at both ends.\n2.  **Try a different monitor port:** If you're using a desktop, try connecting the monitor to a different port on your graphics card (e.g., try a different HDMI or DisplayPort).\n3.  **Update the graphics driver:** Outdated or corrupted graphics drivers can cause screen flickering. You can usually download the latest drivers from the manufacturer's website (NVIDIA, AMD, or Intel).\n4.  **Adjust the screen refresh rate:** Sometimes, an incorrect refresh rate can cause flickering. Go to your display settings and try a different refresh rate.\n\nLet me know if any of these steps resolve the issue!\n"

LlmAgent with Tools

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import FunctionTool


# Tool 1
def get_weather_report(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Returns:
        dict: A dictionary containing the weather information with a 'status' key ('success' or 'error') and a 'report' key with the weather details if successful, or an 'error_message' if an error occurred.
    """
    if city.lower() == "london":
        return {"status": "success", "report": "The current weather in London is cloudy with a temperature of 18 degrees Celsius and a chance of rain."}
    elif city.lower() == "paris":
        return {"status": "success", "report": "The weather in Paris is sunny with a temperature of 25 degrees Celsius."}
    else:
        return {"status": "error", "error_message": f"Weather information for '{city}' is not available."}

weather_tool = FunctionTool(func=get_weather_report)


# Tool 2
def analyze_sentiment(text: str) -> dict:
    """Analyzes the sentiment of the given text.

    Returns:
        dict: A dictionary with 'sentiment' ('positive', 'negative', or 'neutral') and a 'confidence' score.
    """
    if "good" in text.lower() or "sunny" in text.lower():
        return {"sentiment": "positive", "confidence": 0.8}
    elif "rain" in text.lower() or "bad" in text.lower():
        return {"sentiment": "negative", "confidence": 0.7}
    else:
        return {"sentiment": "neutral", "confidence": 0.6}

sentiment_tool = FunctionTool(func=analyze_sentiment)


# Agent
weather_sentiment_agent = Agent(
    model='gemini-2.0-flash-001',
    name='weather_sentiment_agent',
    instruction="""You are a helpful assistant that provides weather information and analyzes the sentiment of user feedback.

**If the user asks about the weather in a specific city, use the 'get_weather_report' tool to retrieve the weather details.**

**If the 'get_weather_report' tool returns a 'success' status, provide the weather report to the user.**

**If the 'get_weather_report' tool returns an 'error' status, inform the user that the weather information for the specified city is not available and ask if they have another city in mind.**

**After providing a weather report, if the user gives feedback on the weather (e.g., 'That's good' or 'I don't like rain'), use the 'analyze_sentiment' tool to understand their sentiment.** Then, briefly acknowledge their sentiment.

You can handle these tasks sequentially if needed.""",
    tools=[weather_tool, sentiment_tool]
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=weather_sentiment_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("What's the weather in london?")

Agent Response:  The current weather in London is cloudy with a temperature of 18 degrees Celsius and a chance of rain.



In [ ]:
call_agent("I don't like rain")

Agent Response:  I understand you're not a fan of rain.



### LLMAgent with FunctionTool, ToolContext and Input/Output Schema - Example 1

In [ ]:
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.tool_context import ToolContext
from pydantic import BaseModel, Field
from typing import List, Optional

# Constants
APP_NAME = "recipe_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "recipe_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"

# --- Mock Data ---
recipes = {
    "pasta carbonara": {
        "ingredients": ["pasta", "eggs", "guanciale", "pecorino romano", "black pepper"],
        "dietary_restrictions": ["none"],
    },
    "chicken tikka masala": {
        "ingredients": ["chicken", "yogurt", "ginger", "garlic", "masala blend"],
        "dietary_restrictions": ["none"],
    },
    "vegan lentil soup": {
        "ingredients": ["lentils", "carrots", "celery", "onion", "vegetable broth"],
        "dietary_restrictions": ["vegan"],
    },
}


# --- Input/Output Schemas ---

class SearchRecipesInput(BaseModel):
    keyword: str = Field(description="The keyword to search for (e.g., ingredient).")

class SearchRecipesOutput(BaseModel):
    recipes: List[str] = Field(description="List of recipe names matching the keyword.")
    message: Optional[str] = Field(default=None, description="Informative message if no recipes are found.")


class CheckDietaryRestrictionsInput(BaseModel):
    recipe_name: str = Field(description="The name of the recipe to check.")
    dietary_restriction: str = Field(description="The dietary restriction to check for (e.g., 'vegan').")

class CheckDietaryRestrictionsOutput(BaseModel):
    is_suitable: bool = Field(description="Whether the recipe is suitable for the given dietary restriction.")
    message: str = Field(description="Informative message about the suitability.")


class GetIngredientListInput(BaseModel):
    recipe_name: str = Field(description="The name of the recipe.")

class GetIngredientListOutput(BaseModel):
    ingredients: List[str] = Field(description="List of ingredients for the recipe.")
    message: Optional[str] = Field(default=None, description="Informative message if the recipe is not found.")

class RecipeAgentInput(BaseModel):
    query: str = Field(description="The user's query to the recipe agent.")

class RecipeAgentOutput(BaseModel):
    response: str = Field(description="The agent's response to the user's query.")


# --- Tools ---

## FunctionTool

# `FunctionTool` is a wrapper that transforms a regular Python function into a tool that an `LlmAgent` can use. It handles:
# - **Function Declaration:**  Automatically generates a function declaration (name, description, parameters) that the LLM understands.  This declaration tells the LLM how to call the function.
# - **Input/Output Conversion:** Converts between the LLM's JSON-based function call arguments and the Python function's arguments, and vice-versa.
# - **Schema Validation:**  (Optional, but highly recommended) Uses `pydantic` schemas to validate the input and output of the function, ensuring type safety and data integrity.

## ToolContext

# `ToolContext` provides contextual information to a tool when it's invoked.  This information is *automatically* injected by the framework.  It includes:
# - **Session Information:**  Access to the current session (user ID, session ID, etc.).
# - **Artifact Management:** Methods for loading and saving artifacts (files, data) associated with the session.
# - **State Access:** Read and modify the session state.
# - **Other Context:**  Information about the current invocation, such as the agent being run.

def search_recipes(keyword: str, tool_context: ToolContext) -> str:
    """Searches for recipes based on a keyword."""
    print(f"Tool Context in search_recipes: {tool_context.__dict__}")
    matching_recipes = [
        recipe_name
        for recipe_name, recipe_data in recipes.items()
        if keyword.lower() in recipe_name.lower() or keyword.lower() in recipe_data["ingredients"]
    ]
    if matching_recipes:
        output = SearchRecipesOutput(recipes=matching_recipes)
        return f"Recipes matching '{keyword}': {', '.join(output.recipes)}."
    else:
        output = SearchRecipesOutput(message=f"No recipes found matching '{keyword}'.")
        return output.message


def check_dietary_restrictions(recipe_name: str, dietary_restriction: str, tool_context: ToolContext) -> str:
    """Checks if a recipe is suitable for a given dietary restriction."""
    print(f"Tool Context in check_dietary_restrictions: {tool_context}")

    recipe_data = recipes.get(recipe_name.lower())
    if recipe_data:
        if dietary_restriction.lower() in recipe_data["dietary_restrictions"]:
            output = CheckDietaryRestrictionsOutput(is_suitable=True, message=f"'{recipe_name}' is suitable for a '{dietary_restriction}' diet.")
            return output.message
        else:
            output = CheckDietaryRestrictionsOutput(is_suitable=False, message=f"'{recipe_name}' is not suitable for a '{dietary_restriction}' diet.")
            return output.message
    else:
        output = CheckDietaryRestrictionsOutput(is_suitable=False, message=f"Recipe '{recipe_name}' not found.")
        return output.message


def get_ingredient_list(recipe_name: str, tool_context: ToolContext) -> str:
    """Returns a list of ingredients for a given recipe."""
    print(f"Tool Context in get_ingredient_list: {tool_context}")

    recipe_data = recipes.get(recipe_name.lower())
    if recipe_data:
        output = GetIngredientListOutput(ingredients=recipe_data['ingredients'])
        return f"Ingredients for '{recipe_name}': {', '.join(output.ingredients)}."
    else:
        output = GetIngredientListOutput(message=f"Recipe '{recipe_name}' not found.")
        return output.message


# Wrap functions with FunctionTool (using the tool's name, description and parameters)
search_recipes_tool = FunctionTool(func=search_recipes)
check_dietary_restrictions_tool = FunctionTool(func=check_dietary_restrictions)
get_ingredient_list_tool = FunctionTool(func=get_ingredient_list)


# --- Agent ---

## LLMAgent

# `LLMAgent` is a specialized agent class designed for interacting with Large Language Models (LLMs).  It handles the complexities of:
# - Sending prompts to the LLM.
# - Receiving responses from the LLM.
# - Detecting and handling function calls (tool use) requested by the LLM.
# - Managing the conversation state.

recipe_agent = LlmAgent(  # Use LlmAgent
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    instruction="""You are a Recipe Agent. Your task is to help users find recipes and check their suitability for dietary restrictions.

    You have access to three tools:
    1. `search_recipes`: Use this tool to find recipes based on a keyword (e.g., ingredient).
    2. `check_dietary_restrictions`: Use this tool to check if a recipe is suitable for a given dietary restriction.
    3. `get_ingredient_list`: Use this tool to get a list of ingredients for a given recipe.

    When a user provides a prompt, first determine what they are asking for.
    - If they are asking for recipes based on a keyword, use the `search_recipes` tool.
    - If they are asking if a recipe is suitable for a dietary restriction, use the `check_dietary_restrictions` tool.
    - If they are asking for a list of ingredients for a recipe, use the `get_ingredient_list` tool.
    Finally, present the information to the user in a clear and concise manner.
    """,
    description="""An agent that can find recipes, check dietary restrictions, and list ingredients.
    It has access to the `search_recipes`, `check_dietary_restrictions`, and `get_ingredient_list` tools.""",
    tools=[search_recipes_tool, check_dietary_restrictions_tool, get_ingredient_list_tool],
    input_schema=RecipeAgentInput,  # Agent-level input schema
    output_schema=RecipeAgentOutput,  # Agent-level output schema
    allow_transfer=False
)

# --- Session and Runner ---
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=recipe_agent, app_name=APP_NAME, session_service=session_service)


# --- Agent Interaction ---

## Input/Output Schemas (Agent Level)

# Defining input and output schemas at the *agent* level (like `RecipeAgentInput` and `RecipeAgentOutput`) provides a high-level contract for how users interact with the agent.  This:
# - Improves the agent's predictability.
# - Makes it easier to integrate the agent into larger systems.
# - Provides a clear interface for the agent.

## Event Types

# The `Runner` generates a stream of `Event` objects that represent different stages of the agent's processing. Understanding these events is key to debugging and controlling the agent's behavior.  The main types are:
# - **Tool Calls:** The LLM requests the use of a specific tool.
# - **Tool Results:** The output produced by a tool.
# - **Agent Responses:** Textual responses generated by the LLM.
# - **Final Response:** The agent's complete, final response.

def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.get_function_calls():  # Check for function calls using the method
            print(f"  LLM Requested Tool Call: {event.get_function_calls()[0].name}")
        elif event.get_function_responses():  # Check for function responses
            print(f"    Tool Result: {event.content.parts[0].text}")
        elif event.content and event.content.role == 'model': # Check for model response
            print(f"  Agent Response: {event.content.parts[0].text}")
        elif event.is_final_response():
            print(f"Agent Final Response: {event.content.parts[0].text}")
        else:
            print(f"  Other Event: {event}")  # For debugging

In [ ]:
call_agent("Find recipes with chicken.")

Tool Context in search_recipes: {'_invocation_context': InvocationContext(artifact_service=None, session_service=<google.adk.sessions.in_memory_session_service.InMemorySessionService object at 0x789c05187650>, memory_service=None, invocation_id='e-9b8bcd03-cb93-4ea5-a185-c4df1199c30c', branch=None, agent=LlmAgent(name='recipe_agent', description='An agent that can find recipes, check dietary restrictions, and list ingredients.\n    It has access to the `search_recipes`, `check_dietary_restrictions`, and `get_ingredient_list` tools.', parent_agent=None, children=[], before_agent_callback=None, after_agent_callback=None, model='gemini-2.0-flash-001', instruction='You are a Recipe Agent. Your task is to help users find recipes and check their suitability for dietary restrictions.\n\n    You have access to three tools:\n    1. `search_recipes`: Use this tool to find recipes based on a keyword (e.g., ingredient).\n    2. `check_dietary_restrictions`: Use this tool to check if a recipe is su

In [ ]:
call_agent("can you give me recipe for it?")

Tool Context in get_ingredient_list: <agents.tools.tool_context.ToolContext object at 0x79671a5be9d0>  LLM Requested Tool Call: get_ingredient_list

    Tool Result: None
  Agent Response: The ingredients for chicken tikka masala are: chicken, yogurt, ginger, garlic, masala blend.


In [ ]:
call_agent("Is pasta carbonara suitable for a vegan diet?")

Tool Context in check_dietary_restrictions: <agents.tools.tool_context.ToolContext object at 0x796719185950>  LLM Requested Tool Call: check_dietary_restrictions

    Tool Result: None
  Agent Response: No, pasta carbonara is not suitable for a vegan diet.



In [ ]:
call_agent("What are the ingredients in vegan lentil soup?")

Tool Context in get_ingredient_list: <agents.tools.tool_context.ToolContext object at 0x796719185950>  LLM Requested Tool Call: get_ingredient_list

    Tool Result: None
  Agent Response: The ingredients for vegan lentil soup are: lentils, carrots, celery, onion, vegetable broth.



**Normal Python Function:**

*   **Direct Execution:** A standard Python function is executed directly when called. You pass arguments, it performs computations, and it returns a value.
*   **No LLM Interaction:** It doesn't inherently interact with Large Language Models (LLMs). It's just plain Python code.
*   **No Automatic Input/Output Handling:** You are responsible for handling input validation, data type conversions, and formatting the output.

**Function Wrapped with `FunctionTool`:**

*   **Indirect Execution via LLM:**  A `FunctionTool` *doesn't* execute the function directly. Instead, it provides a *description* of the function (its name, purpose, parameters, and expected output) to an LLM. The LLM decides *when* and *if* to use the function based on the user's request and the context of the conversation.
*   **LLM-Driven Invocation:** The LLM generates a *function call* (a JSON object) specifying the function to use and the arguments to pass. The `FunctionTool` then:
    *   **Validates Input:** Checks if the arguments provided by the LLM match the function's expected input schema (if defined using pydantic).
    *   **Converts Input:** Transforms the LLM's JSON arguments into the correct Python data types expected by the function.
    *   **Executes the Function:** Calls the underlying Python function with the converted arguments.
    *   **Validates Output:** (Optional) Checks if the function's return value matches the expected output schema.
    *   **Converts Output:** Transforms the Python return value into a JSON-compatible format that the LLM can understand.
    *   **Returns to LLM:** Sends the function's result (as JSON) back to the LLM.
*   **Automatic Declaration:** `FunctionTool` automatically generates the *function declaration* that's sent to the LLM. This declaration is crucial; it's how the LLM knows about the tool's capabilities.  This declaration is derived from the function's signature (name, parameters) and docstring.
* **ToolContext Injection:** It automatically provides a `ToolContext` object to the wrapped function, giving the function access to session information, artifacts, and other contextual data.

**When to Use Which (and Why):**

*   **Use a Normal Python Function:**
    *   **Internal Logic:** When the function is part of your agent's internal logic *and is not intended to be directly accessible to the LLM*.  For example, helper functions, data processing routines within a tool, etc.
    *   **Direct Control:** When you need precise, deterministic control over when and how the function is executed. You call it directly from your code.

*   **Use `FunctionTool`:**
    *   **LLM-Accessible Tools:** When you want to expose a function's capabilities *to the LLM*. This allows the LLM to decide when to use the function to fulfill the user's request.  This is the core concept of tool use with LLMs.
    *   **Dynamic Behavior:** When you want the agent's behavior to be more dynamic and adaptable. The LLM can choose the appropriate tool based on the situation, rather than you hardcoding the tool usage.
    *   **Input/Output Handling:** When you want to leverage automatic input validation, type conversion, and output formatting. This simplifies your code and makes it more robust.  Using `pydantic` schemas with `FunctionTool` is highly recommended for this.
    * **Contextual Information:** When your tool needs access to session-specific data (like user ID, session ID, or artifacts) via the `ToolContext`.

**In essence:**

Think of `FunctionTool` as a bridge between your Python code and the LLM.  It allows the LLM to "see" and "use" your functions as tools.  Normal Python functions are for internal logic; `FunctionTool`-wrapped functions are for exposing capabilities to the LLM.  The developer should use `FunctionTool` whenever they want the LLM to be able to choose to execute a function as part of its response strategy. They should use a normal Python function when they *don't* want the LLM to have direct access to call that function.


### LLMAgent with FunctionTool, ToolContext and Input/Output Schema - Example 2

In [ ]:
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.tool_context import ToolContext
from pydantic import BaseModel, Field
from typing import List, Optional

# --- Input/Output Schemas ---

class SearchRecipesInput(BaseModel):
    keyword: str = Field(description="The keyword to search for (e.g., ingredient).")

class SearchRecipesOutput(BaseModel):
    recipes: List[str] = Field(description="List of recipe names matching the keyword.")
    message: Optional[str] = Field(default=None, description="Informative message if no recipes are found.")

class RecipeAgentInput(BaseModel):
    query: str = Field(description="The user's query to the recipe agent.")

class RecipeAgentOutput(BaseModel):
    response: str = Field(description="The agent's response to the user's query.")

# --- Tools ---

def search_recipes(keyword: str, tool_context: ToolContext) -> str:
    """Searches for recipes based on a keyword."""
    recipes = {
        "pasta carbonara": {
            "ingredients": ["pasta", "eggs", "guanciale", "pecorino romano", "black pepper"],
        },
        "chicken tikka masala": {
            "ingredients": ["chicken", "yogurt", "ginger", "garlic", "masala blend"],
        },
        "vegan lentil soup": {
            "ingredients": ["lentils", "carrots", "celery", "onion", "vegetable broth"],
        },
    }
    matching_recipes = [
        recipe_name
        for recipe_name, recipe_data in recipes.items()
        if keyword.lower() in recipe_name.lower() or keyword.lower() in recipe_data["ingredients"]
    ]
    if matching_recipes:
        output = SearchRecipesOutput(recipes=matching_recipes)
        return f"Recipes matching '{keyword}': {', '.join(output.recipes)}."
    else:
        output = SearchRecipesOutput(message=f"No recipes found matching '{keyword}'.")
        return output.message

# Wrap functions with FunctionTool
search_recipes_tool = FunctionTool(func=search_recipes)

# --- Agent ---
recipe_agent = LlmAgent(
    model="gemini-2.0-flash-001",
    name="recipe_agent",
    instruction="""You are a Recipe Agent. Your task is to help users find recipes.
    You have access to one tool:
    1. `search_recipes`: Use this tool to find recipes based on a keyword (e.g., ingredient).
    When a user provides a prompt, use the `search_recipes` tool to find recipes based on the keyword in the prompt.
    """,
    description="""An agent that can find recipes using the `search_recipes` tool.""",
    allow_transfer=False,
    tools=[search_recipes_tool],
    input_schema=RecipeAgentInput,
    output_schema=RecipeAgentOutput,
)

# --- Session and Runner ---
session_service = InMemorySessionService()
session = session_service.create_session(app_name="recipe_app", user_id="12345", session_id="123344")
runner = Runner(agent=recipe_agent, app_name="recipe_app", session_service=session_service)

# --- Agent Interaction ---

def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id="12345", session_id="123344", new_message=content)

    for event in events:
        if event.get_function_calls():
            print(f"  LLM Requested Tool Call: {event.get_function_calls()[0].name}")
        elif event.get_function_responses():
            print(f"    Tool Result: {event.content.parts[0].text}")
        elif event.content and event.content.role == 'model':
            print(f"  Agent Response: {event.content.parts[0].text}")
        elif event.is_final_response():
            print(f"Agent Final Response: {event.content.parts[0].text}")
        else:
            print(f"  Other Event: {event}")

# Example usage
call_agent("What recipes include pasta?")

  LLM Requested Tool Call: search_recipes
    Tool Result: None
  Agent Response: OK. I found one recipe that includes pasta: pasta carbonara.



###  Quickstart session, states and SessionService

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

# --- Constants ---
APP_NAME = "capital_finder_app"
USER_ID = "quickstart_user"
SESSION_ID = "session_abc"
MODEL = "gemini-2.0-flash-001"

# Agent
capital_agent = LlmAgent(
    model=MODEL,
    name="CapitalFinderAgent",
    instruction="""You are an agent that finds the capital of a given country.
    When asked for the capital, respond *only* with the name of the capital city.
    """,
    output_key="capital_city" # Save the agent's final response text to state['capital_city']
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=capital_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("\nAgent Response: ", final_response)

initial_session = session_service.get_session(APP_NAME, USER_ID, SESSION_ID)
print(f"Initial Session State: {initial_session.state}") # Should be empty {}

call_agent("What is the capital of france?")

final_session = session_service.get_session(APP_NAME, USER_ID, SESSION_ID)
print(f"Final Session State: {final_session.state}") # Should now contain {'capital_city': 'Paris'}

Initial Session State: {}

Agent Response:  Paris

Final Session State: {'capital_city': 'Paris\n'}


### Session State - State Manupilation

In [ ]:
from google.adk.agents import LlmAgent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.events import Event, EventActions

# --- Constants ---
APP_NAME = "task_manager_app"
USER_ID = "test_user"
AGENT_NAME = "task_manager_agent"
MODEL_NAME = "gemini-2.0-flash-001"  # Or any suitable model

# --- Agent Definition ---
#  Simplified instruction, as we're handling logic directly
task_agent = LlmAgent(
    model=MODEL_NAME,
    name=AGENT_NAME,
    instruction="""You are a Task Management Agent. Respond to user requests to manage tasks.
    """,
)


# --- Helper Functions ---

def add_task(session, task_description):
    """Adds a task to the task list."""
    tasks = session.state.get("user:tasks", [])  # Get current tasks (or empty list)
    new_task_id = len(tasks) + 1
    new_task = {"id": new_task_id, "description": task_description, "status": "pending"}
    tasks.append(new_task)
    # Use EventActions to update the state (delta update)
    add_event = Event(author="agent", actions=EventActions(state_delta={"user:tasks": tasks}))
    session_service.append_event(session, add_event)
    return f"Task '{task_description}' added with ID {new_task_id}."

def modify_task(session, task_id, new_status):
    """Modifies the status of a task."""
    tasks = session.state.get("user:tasks", [])
    try:
        task_id = int(task_id)  # Ensure task_id is an integer
    except ValueError:
        return "Invalid task ID. Please provide a number."

    for i, task in enumerate(tasks):
        if task["id"] == task_id:
            tasks[i]["status"] = new_status
            # Update state via EventActions
            modify_event = Event(author='agent', actions=EventActions(state_delta={"user:tasks": tasks}))
            session_service.append_event(session, modify_event)
            return f"Task {task_id} status updated to '{new_status}'."
    return f"Task with ID {task_id} not found."


def delete_task(session, task_id):
    """Deletes a task from the task list."""
    tasks = session.state.get("user:tasks", [])
    try:
        task_id = int(task_id)
    except ValueError:
        return "Invalid task ID.  Please provide a number."

    updated_tasks = [task for task in tasks if task["id"] != task_id]
    if len(updated_tasks) < len(tasks):
        # Update state via EventActions
        delete_event = Event(author='agent', actions=EventActions(state_delta={"user:tasks": updated_tasks}))
        session_service.append_event(session, delete_event)
        return f"Task {task_id} deleted."
    return f"Task with ID {task_id} not found."

def list_tasks(session):
    """Lists all tasks for the user."""
    tasks = session.state.get("user:tasks", [])
    if not tasks:
        return "You have no tasks."
    task_list_str = "\n".join(
        f"{task['id']}: {task['description']} ({task['status']})" for task in tasks
    )
    return f"Your tasks:\n{task_list_str}"


def call_agent(user_input, session):
    """Sends user input to the agent and processes events."""
    content = types.Content(role="user", parts=[types.Part(text=user_input)])
    events = runner.run(user_id=USER_ID, session_id=session.id, new_message=content) #session.id, not SESSION_ID

    final_response_text = ""
    for event in events:
      if event.content and event.content.role == 'model':
          final_response_text = event.content.parts[0].text
          break  # Exit loop after getting the final response.

    return final_response_text

In [ ]:
# 1. Create a Session (with initial state, if any)
#  demonstrates: create_session, InMemorySessionService, initial state
USER_ID = "test_user2"
session_service = InMemorySessionService()

session = session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)  #  Let the service generate the ID

print(f"Created session with ID: {session.id}")
runner = Runner(agent=task_agent, app_name=APP_NAME, session_service=session_service)

Created session with ID: b760e622-b598-4529-8d5e-77e72fd19c43


In [ ]:
session.id

'e834de5e-cbf1-42cb-92a4-a592f6b6bc5e'

In [ ]:
retrieved_session = session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=session.id
)
print(f"\nRetrieved session state:")
retrieved_session.state


Retrieved session state:


{}

In [ ]:
print("\nAdding tasks directly to state via Function...")

add_task(session, "Buy milk")
add_task(session, "Walk the dog")
add_task(session, "Prepare presentation")
add_task(session, "Buy groceries")


Adding tasks directly to state via Function...


"Task 'Buy groceries' added with ID 4."

In [ ]:
# 3. Retrieve and display the session (demonstrates get_session)
retrieved_session = session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=session.id
)
print(f"\nRetrieved session state:")
retrieved_session.state


Retrieved session state:


{'user:tasks': [{'id': 1, 'description': 'Buy milk', 'status': 'pending'},
  {'id': 2, 'description': 'Walk the dog', 'status': 'pending'},
  {'id': 3, 'description': 'Prepare presentation', 'status': 'pending'},
  {'id': 4, 'description': 'Buy groceries', 'status': 'pending'}]}

In [ ]:
# 4. List tasks (demonstrates accessing state)
print("\nListing tasks...")
print(list_tasks(retrieved_session))  # Using a helper, accessing state directly


Listing tasks...
Your tasks:
1: Buy milk (pending)
2: Walk the dog (pending)
3: Prepare presentation (pending)
4: Buy groceries (pending)


In [ ]:
# 5. Modify a task (demonstrates state modification)
print("\nModifying task 2...")
print(modify_task(retrieved_session, "2", "completed"))
retrieved_session = session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=session.id
)
print(f"Modified session state:")
retrieved_session.state


Modifying task 2...
Task 2 status updated to 'completed'.
Modified session state:


{'user:tasks': [{'id': 1, 'description': 'Buy milk', 'status': 'pending'},
  {'id': 2, 'description': 'Walk the dog', 'status': 'completed'},
  {'id': 3, 'description': 'Prepare presentation', 'status': 'pending'},
  {'id': 4, 'description': 'Buy groceries', 'status': 'pending'}]}

In [ ]:
# 6. Delete a task (demonstrates state deletion)
print("\nDeleting task 1...")
print(delete_task(retrieved_session, "1"))
retrieved_session = session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=session.id
)


Deleting task 1...
Task 1 deleted.


In [ ]:
print(f"Session state after deletion:")
retrieved_session.state

Session state after deletion:


{'user:tasks': [{'id': 2,
   'description': 'Walk the dog',
   'status': 'completed'},
  {'id': 3, 'description': 'Prepare presentation', 'status': 'pending'},
  {'id': 4, 'description': 'Buy groceries', 'status': 'pending'}]}

In [ ]:
# 7. List sessions for the user (demonstrates list_sessions)
# Demonstrates:  list_sessions
print(f"\nSessions for user {USER_ID}:")
session_service.list_sessions(APP_NAME, USER_ID)


Sessions for user test_user2:


ListSessionsResponse(session_ids=['b760e622-b598-4529-8d5e-77e72fd19c43'])

In [ ]:
# 8. Delete a session (demonstrates delete_session).
session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)
print(f"\nDeleted session: {session.id}")
retrieved_session = session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)


Deleted session: e834de5e-cbf1-42cb-92a4-a592f6b6bc5e


In [ ]:
if retrieved_session: # Should be None.
    print(retrieved_session)
else:
    print("Session deleted successfully, as could not be retrieved.")

Session deleted successfully, as could not be retrieved.


### Session State Example 1

In [ ]:
from google.adk.sessions import InMemorySessionService

# Create the session service
session_service = InMemorySessionService()

# Create a session (with initial state)
session = session_service.create_session(
    app_name="my_app",
    user_id="user1",
    state={"order_status": "pending", "items": ["shirt"]},
)

# --- Direct State Manipulation (Generally NOT Recommended) ---

# 1. Retrieve the session (to get the current state)
retrieved_session = session_service.get_session(
    app_name="my_app", user_id="user1", session_id=session.id
)

# 2. Access the state dictionary directly
current_state = retrieved_session.state

# 3. Modify the state directly
current_state["order_status"] = "shipped"  # Update existing key
current_state["tracking_number"] = "XYZ123"  # Add new key
current_state["items"] = ["shirt", "pants"]   #Modify existing key.

# 4.  The changes are reflected *immediately* in the retrieved session.
print(retrieved_session.state)
# Output will show:
# {'order_status': 'shipped', 'items': ['shirt', 'pants'], 'tracking_number': 'XYZ123'}

# --- Deleting a Key (Directly) ---

# 1. Access the state dictionary
current_state = retrieved_session.state

# 2. Delete the key using 'del' or set to None.
del current_state["tracking_number"]  # Or: current_state["tracking_number"] = None
# del current_state["order_status"] # Example showing how to delete completely.


# 3. Verify the deletion
print(retrieved_session.state)  # 'tracking_number' is gone
# Output
# {'order_status': 'shipped', 'items': ['shirt', 'pants']}


# --- Demonstrating State Prefixes (Directly) ---

current_state["app:max_retries"] = 3  # Adding an app-level setting
current_state["user:preferred_contact"] = "email"  # User-specific setting
current_state["temp:request_id"] = "temp_val"       # Temporary value

print(retrieved_session.state)

retrieved_session2 = session_service.get_session(
    app_name="my_app", user_id="user1", session_id=session.id
)

print(retrieved_session2.state)  # 'temp:request_id' will *not* be present
# Demonstrating how to access:
print(session_service.app_state)
print(session_service.user_state)

{'order_status': 'shipped', 'items': ['shirt', 'pants'], 'tracking_number': 'XYZ123'}
{'order_status': 'shipped', 'items': ['shirt', 'pants']}
{'order_status': 'shipped', 'items': ['shirt', 'pants'], 'app:max_retries': 3, 'user:preferred_contact': 'email', 'temp:request_id': 'temp_val'}
{'order_status': 'pending', 'items': ['shirt']}
{}
{'task_manager_app': {'test_user': {'tasks': [{'id': 1, 'description': 'Buy milk', 'status': 'pending'}]}, 'test_user2': {'tasks': [{'id': 2, 'description': 'Walk the dog', 'status': 'completed'}, {'id': 3, 'description': 'Prepare presentation', 'status': 'pending'}, {'id': 4, 'description': 'Buy groceries', 'status': 'pending'}]}}}


### Session State - delta_states

In [ ]:
from google.adk.sessions import InMemorySessionService


# Create the session service
session_service = InMemorySessionService()

# Create a session (with initial state)
session = session_service.create_session(
    app_name="my_app",
    user_id="user1",
    state={"order_status": "pending", "items": ["shirt"], "notes": "Initial order"},
)

# --- Direct State Manipulation (Generally NOT Recommended) ---

# 1. Retrieve the session (to get the current state)
retrieved_session = session_service.get_session(
    app_name="my_app", user_id="user1", session_id=session.id
)

# 2. Access the state dictionary directly
current_state = retrieved_session.state

# --- Simulating a Delta Update (Directly) ---

# Directly modify the state dictionary as if applying a state_delta:
current_state["order_status"] = "shipped"  # Update existing key
current_state["tracking_number"] = "XYZ456"  # Add a new key
current_state["items"].append("pants")      # Modify a list (append)
del current_state["notes"]                   # Delete a key

# The changes are reflected *immediately* in the retrieved_session (with InMemorySessionService)
print(retrieved_session.state)

# --- Demonstrating State Prefixes (Directly) ---
# We need to use setdefault to properly initialize nested dictionaries.
session_service.app_state.setdefault("my_app", {})["max_retries"] = 3  # app-level
session_service.user_state.setdefault("my_app", {}).setdefault("user1", {})["pref_contact"] = "email"
current_state["temp:request_id"] = "temp_value"

print("\nState with prefixes added:")
print(retrieved_session.state) #temp wont be there

retrieved_session = session_service.get_session(
    app_name="my_app", user_id="user1", session_id=session.id
)
print("\nState after retrieval (temp should be gone):")

print(retrieved_session.state)

print("\nApp State:")
print(session_service.app_state)
print("\nUser State:")
print(session_service.user_state)

{'order_status': 'shipped', 'items': ['shirt', 'pants'], 'tracking_number': 'XYZ456'}

State with prefixes added:
{'order_status': 'shipped', 'items': ['shirt', 'pants'], 'tracking_number': 'XYZ456', 'temp:request_id': 'temp_value'}

State after retrieval (temp should be gone):
{'order_status': 'pending', 'items': ['shirt'], 'notes': 'Initial order', 'app:max_retries': 3, 'user:pref_contact': 'email'}

App State:
{'my_app': {'max_retries': 3}}

User State:
{'task_manager_app': {'test_user': {'tasks': [{'id': 1, 'description': 'Buy milk', 'status': 'pending'}]}, 'test_user2': {'tasks': [{'id': 2, 'description': 'Walk the dog', 'status': 'completed'}, {'id': 3, 'description': 'Prepare presentation', 'status': 'pending'}, {'id': 4, 'description': 'Buy groceries', 'status': 'pending'}]}}, 'my_app': {'user1': {'pref_contact': 'email'}}}


In [ ]:
from google.adk.sessions import InMemorySessionService

# Create the session service and session (as before)
session_service = InMemorySessionService()
session = session_service.create_session(
    app_name="my_app",
    user_id="user1",
    state={"order_status": "pending", "items": ["shirt", "hat"], "notes": "Initial order"},
)
retrieved_session = session_service.get_session(
    app_name="my_app", user_id="user1", session_id=session.id
)
current_state = retrieved_session.state

# --- Simulate a Delta Update (Directly) ---
current_state["order_status"] = "shipped"  # Update
current_state["tracking_number"] = "XYZ456"  # Add
current_state["items"].append("socks")      # Modify (append to list)
del current_state["notes"]                 # Delete

print(retrieved_session.state)

{'order_status': 'shipped', 'items': ['shirt', 'hat', 'socks'], 'app:max_retries': 3, 'user:pref_contact': 'email', 'tracking_number': 'XYZ456'}


### Accessing Session Properties

In [ ]:
from google.adk.sessions import InMemorySessionService


# Create a simple session to examine its properties
temp_service = InMemorySessionService()
example_session = temp_service.create_session(
    app_name="my_app",
    user_id="example_user",
    state={"initial_value": 1}
)

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {example_session.id}")   # Unique identifier
print(f"Application Name (`app_name`): {example_session.app_name}") # Which app it belongs to
print(f"User ID (`user_id`):         {example_session.user_id}")    # Who the user is
print(f"State (`state`):           {example_session.state}")       # The dynamic 'notes' dictionary
print(f"Events (`events`):         {example_session.events}")      # The conversation history (initially empty)
print(f"Last Update (`last_update_time`): {example_session.last_update_time:.2f}") # When it was last triggered
print(f"---------------------------------")

--- Examining Session Properties ---
ID (`id`):                0f0e38f1-fc71-47f2-9c07-b78bf8c04bd6
Application Name (`app_name`): my_app
User ID (`user_id`):         example_user
State (`state`):           {'initial_value': 1}
Events (`events`):         []
Last Update (`last_update_time`): 1743134817.66
---------------------------------


### Using InMemorySessionService Methods

In [ ]:
# Example: Using InMemorySessionService Methods
from google.adk.sessions import InMemorySessionService
from google.adk.events import Event, EventActions
from google.genai import types
import time
import uuid

print("\n--- Demonstrating InMemorySessionService ---")

# 1. Instantiate
session_service = InMemorySessionService()
app_name, user_id = "memory_app", "user_mem"
session_id = "mem_session_1"

# 2. Create Session
current_session = session_service.create_session(
    app_name=app_name, user_id=user_id, session_id=session_id, state={"counter": 0}
)
print(f"Created Session: ID={current_session.id}, State={current_session.state}")

# 3. Append Event with State Delta
user_event = Event(
    invocation_id="inv_1", author="user", content=types.Content(parts=[types.Part(text="Increment")])
)
session_service.append_event(current_session, user_event)  # No state change yet

agent_event = Event(
    invocation_id="inv_2", author="agent",
    actions=EventActions(state_delta={"counter": 1})  # Increment counter
)
session_service.append_event(current_session, agent_event)
print(f"Appended Event, state['counter'] should be 1")

# 4. Get Session
retrieved_session = session_service.get_session(app_name, user_id, session_id)
print(f"Retrieved Session: ID={retrieved_session.id}, State={retrieved_session.state}")
print(f"Events in session: {len(retrieved_session.events)}")  # Shows 2 events were added

# 5. List Sessions
session_list = session_service.list_sessions(app_name, user_id)
print(f"List Sessions for {user_id}: {session_list.session_ids}")

# 6. Delete Session
session_service.delete_session(app_name, user_id, session_id)
print(f"Deleted Session: {session_id}")

# 7. Get Session (should fail)
deleted_session = session_service.get_session(app_name, user_id, session_id)
print(f"Retrieve after delete: {'Session found (unexpected!)' if deleted_session else 'Session not found (correct)'}")
print("------------------------------------------")


--- Demonstrating InMemorySessionService ---
Created Session: ID=mem_session_1, State={'counter': 0}
Appended Event, state['counter'] should be 1
Retrieved Session: ID=mem_session_1, State={'counter': 1}
Events in session: 2
List Sessions for user_mem: ['mem_session_1']
Deleted Session: mem_session_1
Retrieve after delete: Session not found (correct)
------------------------------------------


###  Using DatabaseSessionService Methods (with SQLite for demo)

In [ ]:
# Example: Using DatabaseSessionService Methods (with SQLite for demo)

# NOTE: Requires `sqlalchemy` to be installed.
# NOTE: This creates a file 'db_sessions_demo.db' in the current directory.
from google.adk.sessions import DatabaseSessionService
from google.adk.events import Event, EventActions
from google.genai import types # Make sure types is imported
import time
import uuid
import os # To manage the demo database file

print("\n--- Demonstrating DatabaseSessionService (SQLite) ---")
DB_FILE = "./db_sessions_demo.db" # Define path for the database file
DB_DIR = os.path.dirname(DB_FILE) # Get directory path

# Ensure the directory exists (useful if DB_FILE includes subdirectories)
if DB_DIR and not os.path.exists(DB_DIR):
    os.makedirs(DB_DIR)
    print(f"Created directory: {DB_DIR}")

# Remove the database file if it exists from a previous run for a clean demo
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"Removed existing demo DB file: {DB_FILE}")

# 1. Instantiate (using SQLite file)
# The DatabaseSessionService's __init__ method will handle DB/table creation.
db_service = DatabaseSessionService(db_url=f"sqlite:///{DB_FILE}")
print(f"Instantiated DatabaseSessionService. DB file '{DB_FILE}' ensured/created.")

APP_DB, USER_DB = "db_app", "user_db"
SESSION_ID_DB = "db_session_1"

# 2. Create Session
session_db = db_service.create_session(
    app_name=APP_DB, user_id=USER_DB, session_id=SESSION_ID_DB, state={"status": "new"}
)
print(f"Created Session: ID={session_db.id}, State={session_db.state}")

# 3. Append Event with State Delta
# *** FIX: Ensure event has a 'content' object, even if minimal ***
event_db_1 = Event(
    invocation_id="inv_db1", author="agent",
    content=types.Content(parts=[types.Part(text="System update: Processing")]), # Add content
    actions=EventActions(state_delta={"status": "processing", "db_key": "db_val"})
)
# Note: append_event updates the state in the DB and the passed session's last_update_time
db_service.append_event(session_db, event_db_1)
print(f"Appended Event, state should be updated in the database.")

# 4. Get Session (re-fetch from DB to see persisted changes)
# Note: Must re-fetch session to see DB changes reflected in the object state
retrieved_session_db = db_service.get_session(APP_DB, USER_DB, SESSION_ID_DB)
print(f"Retrieved Session: ID={retrieved_session_db.id}, State={retrieved_session_db.state}")
# Note: Events are not automatically loaded by get_session in this implementation by default.

# 5. List Sessions
sessions_list_db = db_service.list_sessions(APP_DB, USER_DB)
print(f"List Sessions for {USER_DB}: {sessions_list_db.session_ids}")

# 6. List Events (Not Implemented in DatabaseSessionService base implementation)
try:
    db_service.list_events(APP_DB, USER_DB, SESSION_ID_DB)
except NotImplementedError as e:
    print(f"List Events: Received NotImplementedError (as expected in base class)")
except AttributeError as e:
     print(f"List Events: Method not found or not implemented (as expected in base class)")

# 7. Delete Session
db_service.delete_session(APP_DB, USER_DB, SESSION_ID_DB)
print(f"Deleted Session: {SESSION_ID_DB}")

# 8. Get Session (should fail)
deleted_session_check_db = db_service.get_session(APP_DB, USER_DB, SESSION_ID_DB)
print(f"Retrieve after delete: {'Session found (unexpected!)' if deleted_session_check_db else 'Session not found (correct)'}")

# Cleanup demo file (optional, good practice for demos)
# if os.path.exists(DB_FILE):
#     os.remove(DB_FILE)
#     print(f"Cleaned up demo DB file: {DB_FILE}")
print("--------------------------------------------")


--- Demonstrating DatabaseSessionService (SQLite) ---
Removed existing demo DB file: ./db_sessions_demo.db
Instantiated DatabaseSessionService. DB file './db_sessions_demo.db' ensured/created.
Created Session: ID=db_session_1, State={'status': 'new'}
Appended Event, state should be updated in the database.
Retrieved Session: ID=db_session_1, State={'status': 'processing', 'db_key': 'db_val'}
List Sessions for user_db: ['db_session_1']
Deleted Session: db_session_1
Retrieve after delete: Session not found (correct)
--------------------------------------------


### LlMAgent with AntrhopicVertex

In [ ]:
import os
import asyncio
import warnings

# --- GCP/Vertex AI Configuration ---
# Make sure these are set in your environment OR uncomment and set here
# os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-east5" # Make sure its us-east5 or europe-west1

# --- ADK Imports ---
from google.adk.agents import Agent, LlmAgent # Using LlmAgent directly
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from google.adk.models.anthropic_llm import Claude
from google.adk.models.registry import LLMRegistry

# Manually register the Claude model class with the registry
# This step is crucial if the framework doesn't do it automatically
LLMRegistry.register(Claude)


# --- Constants ---
APP_NAME = "anthropic_capital_app"
USER_ID = "anthropic_user"
SESSION_ID = "anthropic_session_1"
AGENT_NAME = "claude_capital_agent"
# --- Anthropic Model Name (via Vertex AI) ---
# Use the model identifier available in your Vertex AI region.
# Example: Claude 3.5 Sonnet. Check Vertex AI documentation for available models.
ANTHROPIC_MODEL = "claude-3-7-sonnet@20250219"
# You might need to adjust this based on availability in your specific GCP project/location.
# Other possibilities: "claude-3-haiku@...", "claude-3-opus@..."

# --- Agent Definition ---
# Simplest agent: takes user input, uses the LLM directly to answer.
capital_agent = LlmAgent(
    model=ANTHROPIC_MODEL, # Specify the Anthropic model identifier
    name=AGENT_NAME,
    instruction="You are a helpful assistant. When asked for the capital of a country, provide only the name of the capital city.",
    description="An agent that provides the capital city of a country using an Anthropic model.",
    # No tools needed for this simple task
    tools=[],
    allow_transfer=False # Keep it simple
)

# --- Session and Runner Setup ---
session_service = InMemorySessionService()
# Ensure session is created before running
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=capital_agent, app_name=APP_NAME, session_service=session_service)

# --- Agent Interaction Logic ---
# Using async version as it's preferred
async def call_agent_async(query):
  print(f"\nUser Query: {query}")
  content = types.Content(role='user', parts=[types.Part(text=query)])
  final_response_text = "Agent did not produce a final response."
  try:
      async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
          if event.is_final_response() and event.content and event.content.parts:
              final_response_text = event.content.parts[0].text
              print(f"Agent Response: {final_response_text}")
              # Break after final response for simplicity in this example
              break
          elif event.error_message:
              final_response_text = f"Agent Error: {event.error_message}"
              print(final_response_text)
              break
  except Exception as e:
      print(f"An error occurred during agent execution: {e}")
      final_response_text = f"Execution Error: {e}"

  return final_response_text

# --- Example Usage ---
async def run_example():
    print("--- Running Anthropic Agent Example ---")
    print(f"Using model: {ANTHROPIC_MODEL}")
    print(f"Make sure GCP Project '{os.environ['GOOGLE_CLOUD_PROJECT']}' and Location '{os.environ['GOOGLE_CLOUD_LOCATION']}' are correct and the model is available there.")
    print("Ensure you are authenticated with GCP (e.g., `gcloud auth application-default login`).")

    await call_agent_async("What is the capital of France?")
    await call_agent_async("What's the capital of Japan?")
    await call_agent_async("Tell me the capital of Germany.")
    print("--- Example Finished ---")

await run_example()

--- Running Anthropic Agent Example ---
Using model: claude-3-5-sonnet-v2@20241022
Make sure GCP Project 'lavi-llm-experiment' and Location 'us-east5' are correct and the model is available there.
Ensure you are authenticated with GCP (e.g., `gcloud auth application-default login`).

User Query: What is the capital of France?
Agent Response: Paris

User Query: What's the capital of Japan?
Agent Response: Tokyo

User Query: Tell me the capital of Germany.
Agent Response: Berlin
--- Example Finished ---


### LlmAgent with Artifact Service

In [ ]:
!pip install pymupdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 23.1 MB/s eta 0:00:00


In [ ]:
# --- Imports ---
import os
import fitz # Still needed if you want to verify the local file exists or for other ops
from google.adk.agents import LlmAgent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Import InMemoryArtifactService
from google.adk.runners import Runner
# We no longer need the custom load_document tool
# from google.adk.tools.function_tool import FunctionTool
# from google.adk.tools.tool_context import ToolContext
from google.adk.tools import load_artifacts_tool # Import the built-in tool
from pydantic import BaseModel, Field
from typing import Optional, List

# --- Constants ---
APP_NAME = "doc_qa_app"
USER_ID = "doc_user_1"
SESSION_ID = "doc_session_1"
AGENT_NAME = "doc_qa_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001" # Or "gemini-1.5-flash-latest"

# --- Actual File Paths ---
# Make sure these paths are correct for your environment
LOCAL_PDF_PATH = "/content/multimodal-finanace-qa_data_unstructured_production_blogpost_Hex-LLM on TPUs in Vertex AI Model Garden _ Google Cloud Blog.pdf"
GCS_PDF_URI = "gs://mlops-for-genai/multimodal-finanace-qa/data/unstructured/prototype/google_cloud_tpu_blog_training.pdf"



# --- Agent Definition ---
# We remove the load_document tool as loading happens externally now
doc_qa_agent = LlmAgent(
    model=GEMINI_2_FLASH,
    name=AGENT_NAME,
    description="An agent that answers questions based on provided PDF documents.",
    # Updated instructions reflecting pre-loaded artifacts
    instruction=f"""You are a Document Question Answering Agent. Your goal is to answer user questions based *only* on the content of PDF documents that have been loaded as artifacts.

Documents are already available as artifacts named 'local_doc' and 'gcs_doc'.

Follow these steps when the user asks a question:
1.  Determine which artifact ('local_doc' or 'gcs_doc') is relevant to the user's question. If the user doesn't specify, you might need to ask or infer based on the question content.
2.  Use the `load_artifacts` tool to load the content of the relevant artifact(s) into your context. Provide the artifact name(s) you determined in step 1.
3.  Once the artifact content is loaded (you'll receive it as a tool response), answer the user's question based *strictly* on the provided text from the artifact.
4.  If the answer isn't found in the artifact, state that clearly. Do not use your general knowledge.
5.  You can also use the artifact service tools implicitly provided to list artifacts if needed.
""",
    tools=[
        load_artifacts_tool # Only need the built-in load_artifacts tool now
        ],
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=1024,
        temperature=0.2
    ),
)

# --- Session, Artifact Service, and Runner Setup ---
session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService() # Use the in-memory artifact service

# --- Pre-loading Artifacts Directly ---
print("\n=== Pre-loading Artifacts ===")
try:
    # 1. Load Local PDF
    print(f"Loading local file: {LOCAL_PDF_PATH}")
    with open(LOCAL_PDF_PATH, "rb") as f:
        local_pdf_bytes = f.read()
    local_part = types.Part.from_bytes(data=local_pdf_bytes, mime_type='application/pdf')
    local_version = artifact_service.save_artifact(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        filename='local_doc', # Assign the name the agent will use
        artifact=local_part
    )
    print(f"  Saved 'local_doc' (version {local_version}) to artifact service.")

    # 2. Load GCS PDF
    # Ensure your Colab environment is authenticated if the URI isn't public
    # from google.colab import auth
    # auth.authenticate_user() # Uncomment if needed
    print(f"Loading GCS file: {GCS_PDF_URI}")
    gcs_part = types.Part.from_uri(uri=GCS_PDF_URI, mime_type='application/pdf')
    gcs_version = artifact_service.save_artifact(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        filename='gcs_doc', # Assign the name the agent will use
        artifact=gcs_part
    )
    print(f"  Saved 'gcs_doc' (version {gcs_version}) to artifact service.")

except FileNotFoundError as e:
    print(f"Error pre-loading artifact: {e}")
    exit() # Stop if files aren't found
except Exception as e:
    print(f"An unexpected error occurred during pre-loading: {e}")
    # Depending on the error (e.g., GCS auth), you might want to exit or continue
    exit()

print("=== Artifact Pre-loading Complete ===\n")


# *** Crucial Step: Pass the SAME artifact_service instance to the Runner ***
runner = Runner(
    agent=doc_qa_agent,
    app_name=APP_NAME,
    session_service=session_service,
    artifact_service=artifact_service # Make sure runner uses the service with pre-loaded data
)

# Create the session (state is initially empty, artifacts are separate)
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)

# --- Agent Interaction Function (Modified slightly for clarity) ---
def call_agent(query):
  print(f"\n--- User Query ---\n{query}")
  content = types.Content(role='user', parts=[types.Part(text=query)])
  final_response_text = "Agent did not provide a final text response."
  print("\n--- Agent Execution Trace ---")

  try:
      events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

      for event in events:
          if event.content:
              if event.get_function_calls():
                  fc = event.get_function_calls()[0]
                  print(f"  [LLM -> Tool Request] Function Call: {fc.name}({fc.args})")
              elif event.get_function_responses():
                   fr = event.get_function_responses()[0]
                   response_str = str(fr.response)
                   if len(response_str) > 300: # Increased truncation length
                       response_str = response_str[:300] + "... (truncated)"
                   print(f"  [Tool -> LLM Response] Function Result ({fr.name}): {response_str}")
              elif event.content.parts[0].text:
                    print(f"  [LLM -> User] Agent Thought/Response: {event.content.parts[0].text.strip()}")
                    if event.is_final_response():
                         final_response_text = event.content.parts[0].text.strip()
              else:
                  print(f"  [Event] Other content: {event.content}")
          elif event.actions:
               print(f"  [Event] Actions: {event.actions}")
          else:
               print(f"  [Event] Other event type: {type(event)}")

  except Exception as e:
      print(f"\n--- Error during agent execution ---")
      print(e)
      # Attempt to retrieve the latest state even on error
      retrieved_session = session_service.get_session(APP_NAME, USER_ID, SESSION_ID)
      print("Session state on error:", retrieved_session.state if retrieved_session else "N/A")
      final_response_text = f"An error occurred: {e}"

  print(f"\n--- Final Agent Response ---")
  print(final_response_text)
  print("-" * (len("--- Final Agent Response ---")))


Verified local PDF exists: /content/multimodal-finanace-qa_data_unstructured_production_blogpost_Hex-LLM on TPUs in Vertex AI Model Garden _ Google Cloud Blog.pdf


ValidationError: 2 validation errors for LlmAgent
tools.0.callable
  Input should be callable [type=callable_type, input_value=<module 'google.adk.tools...load_artifacts_tool.py'>, input_type=module]
    For further information visit https://errors.pydantic.dev/2.11/v/callable_type
tools.0.is-instance[BaseTool]
  Input should be an instance of BaseTool [type=is_instance_of, input_value=<module 'google.adk.tools...load_artifacts_tool.py'>, input_type=module]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of

In [ ]:

# --- Demonstrating Artifact Service Methods & Agent Interaction ---

print("\n=== Demonstrating Artifact Service After Pre-load ===\n")

# 1. List Artifact Keys (Verify pre-load)
print("1. Listing available artifacts...")
current_artifacts = artifact_service.list_artifact_keys(APP_NAME, USER_ID, SESSION_ID)
print(f"  Current artifact keys: {current_artifacts}") # Should show ['gcs_doc', 'local_doc']

# 2. List Versions
print("\n2. Listing versions for 'local_doc'...")
local_doc_versions = artifact_service.list_versions(APP_NAME, USER_ID, SESSION_ID, 'local_doc')
print(f"  Versions for 'local_doc': {local_doc_versions}")

# 3. Ask agent questions (Should trigger load_artifacts tool)
print("\n3. Asking a question about the local document (Hex LLM Blog)...")
call_agent("What is this blog post about? Use the local_doc artifact.")

print("\n4. Asking a question about the GCS document (TPU Blog)...")
call_agent("What is the main topic of the document named 'gcs_doc'?")

# 4. Delete Artifact
print("\n5. Deleting 'local_doc' artifact...")
artifact_service.delete_artifact(APP_NAME, USER_ID, SESSION_ID, 'local_doc')
print("  'local_doc' deleted.")
current_artifacts_after_delete = artifact_service.list_artifact_keys(APP_NAME, USER_ID, SESSION_ID)
print(f"  Current artifact keys after delete: {current_artifacts_after_delete}")

print("\n6. Trying to ask about the deleted document...")
call_agent("Summarize the local_doc artifact for me.") # Agent should state it cannot find 'local_doc'

print("\n=== End of Demonstration ===\n")

In [ ]:
# --- Demonstrating Artifact Service Methods ---

print("\n=== Demonstrating Artifact Service ===\n")

# 1. Save Artifact (via the load_document tool)
print("1. Loading local document...")
call_agent(f"Load the document at '{DUMMY_LOCAL_PDF_PATH}' and call it 'local_doc'.")

print("\n2. Loading GCS document...")
# Ensure your Colab environment is authenticated to access GCS if the URI isn't public
# from google.colab import auth
# auth.authenticate_user()
call_agent(f"Load the document from '{GCS_PDF_URI}' and name it 'gcs_doc'.")

# 2. List Artifact Keys
print("\n3. Listing available artifacts...")
current_artifacts = artifact_service.list_artifact_keys(APP_NAME, USER_ID, SESSION_ID)
print(f"  Current artifact keys: {current_artifacts}")

# 3. List Versions
print("\n4. Listing versions for 'local_doc'...")
local_doc_versions = artifact_service.list_versions(APP_NAME, USER_ID, SESSION_ID, 'local_doc')
print(f"  Versions for 'local_doc': {local_doc_versions}")

# 4. Load Artifact (via the agent asking a question)
print("\n5. Asking a question about the local document...")
call_agent("What is the dummy local document about?") # This should trigger load_artifacts tool

print("\n6. Asking a question about the GCS document...")
# Note: The content of the public GCS PDF (2403.05530.pdf) is about "Gemini 1.5".
call_agent("What model is discussed in the 'gcs_doc' artifact?")

# 5. Delete Artifact
print("\n7. Deleting 'local_doc' artifact...")
artifact_service.delete_artifact(APP_NAME, USER_ID, SESSION_ID, 'local_doc')
print("  'local_doc' deleted.")
current_artifacts_after_delete = artifact_service.list_artifact_keys(APP_NAME, USER_ID, SESSION_ID)
print(f"  Current artifact keys after delete: {current_artifacts_after_delete}")

print("\n8. Trying to ask about the deleted document...")
call_agent("Tell me again, what was the local document about?") # Agent should state it cannot find the artifact

print("\n=== End of Demonstration ===\n")

In [ ]:
# --- Setup: Install necessary libraries ---
# pypdf is used to read local PDF bytes for the from_bytes example
!pip install google-cloud-aiplatform google-adk pypdf --quiet

# --- ADK Imports ---
import os
import asyncio
import warnings
import mimetypes
from typing import Any, Optional, Dict, List, AsyncGenerator

# Use pypdf for reading local PDF bytes
try:
    import pypdf
except ImportError:
    print("Please install pypdf: !pip install pypdf")
    pypdf = None # Set to None if import fails

from google.adk.agents import Agent, LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.adk.artifacts import BaseArtifactService, InMemoryArtifactService, GcsArtifactService
from google.adk.tools import load_artifacts # Import the specific tool
from google.genai import types


# --- Constants ---
APP_NAME_BASE = "pdf_summarizer_app"
USER_ID = "pdf_user"
AGENT_NAME = "pdf_summarizer_agent"
MODEL_NAME = "gemini-2.0-flash-001" # Changed to generally available model

# --- Helper Function to Save PDF Artifact ---
def save_pdf_artifact(
    file_path_or_uri: str,
    artifact_service: BaseArtifactService,
    app_name: str,
    user_id: str,
    session_id: str,
    save_as_filename: Optional[str] = None
) -> Optional[str]:
    """Creates a Part (from_uri or from_bytes) and saves it using the artifact service."""
    print(f"  Processing PDF: {file_path_or_uri}")
    actual_filename = os.path.basename(file_path_or_uri) if not save_as_filename else save_as_filename
    pdf_part: Optional[types.Part] = None
    mime_type = "application/pdf" # Explicitly PDF

    try:
        if file_path_or_uri.startswith("gs://"):
            # Create a Part referencing the GCS URI
            pdf_part = types.Part.from_uri(uri=file_path_or_uri, mime_type=mime_type)
            print(f"    Created Part from URI.")

        else:
            # Local File: Read bytes and create Part from bytes
            if not pypdf:
                 print("    Error: pypdf library is required to read local PDF bytes but not found.")
                 return None
            if not os.path.exists(file_path_or_uri):
                print(f"    Error: Local PDF file not found: {file_path_or_uri}")
                return None

            # --- Reading Local PDF Bytes ---
            # Note: For summarization, sending extracted text is often better for the LLM.
            # However, to demonstrate from_bytes with the *raw PDF*, we read the raw bytes.
            # The LLM needs multimodal capabilities to process raw PDF bytes directly.
            # Gemini 2 Flash might struggle with raw PDF bytes. Gemini 1.5 Pro is better.
            # Consider extracting text first if using a model less capable with raw PDFs.
            with open(file_path_or_uri, "rb") as f:
                 pdf_bytes = f.read()
            pdf_part = types.Part.from_bytes(data=pdf_bytes, mime_type=mime_type)
            print(f"    Created Part from bytes ({len(pdf_bytes)} bytes).")
            # --- End Reading Local PDF Bytes ---

            # --- Alternative: Extract Text from Local PDF (Often better for LLM) ---
            # pdf_reader = pypdf.PdfReader(file_path_or_uri)
            # extracted_text = "".join(page.extract_text() or "" for page in pdf_reader.pages)
            # if not extracted_text:
            #     print(f"    Warning: Could not extract text from local PDF: {file_path_or_uri}")
            #     return None
            # text_bytes = extracted_text.encode('utf-8')
            # pdf_part = types.Part.from_bytes(data=text_bytes, mime_type='text/plain') # Send as text
            # print(f"    Created Part from extracted text ({len(text_bytes)} bytes).")
            # --- End Alternative ---


        if pdf_part:
            version = artifact_service.save_artifact(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
                filename=actual_filename,
                artifact=pdf_part
            )
            print(f"    Saved artifact '{actual_filename}' as version {version}.")
            return actual_filename
        else:
             print(f"    Error: Could not create Part for {file_path_or_uri}")
             return None

    except Exception as e:
        print(f"    Error processing file {file_path_or_uri}: {e}")
        return None

# --- Agent Definition ---
pdf_summarizer_agent = LlmAgent(
    model=MODEL_NAME,
    name=AGENT_NAME,
    instruction=f"""You are a PDF document summarization assistant.
You will be given the filename of a PDF saved as an artifact.
Your task is to:
1. Use the `{load_artifacts.name}` tool to load the content of the specified PDF artifact.
2. Read the content provided by the tool (which might be the full text or the raw PDF data depending on the model's capability).
3. Provide a concise summary of the document.
If you cannot access or process the document, state that clearly.
""",
    description="Summarizes PDF documents provided as artifacts.",
    tools=[load_artifacts], # Make the tool available
    allow_transfer=False,
)

# --- Agent Interaction Logic ---
async def run_pdf_summarization(
        artifact_service: BaseArtifactService,
        session_service: InMemorySessionService,
        app_name: str,
        session_id: str,
        pdf_path_or_uri: str):
    """Saves the PDF as an artifact and runs the summarizer agent."""

    print(f"\n--- Starting PDF Summarization using {artifact_service.__class__.__name__} ---")
    session = session_service.create_session(app_name=app_name, user_id=USER_ID, session_id=session_id)
    runner = Runner(agent=pdf_summarizer_agent, app_name=app_name, session_service=session_service, artifact_service=artifact_service)

    # 1. Save PDF as artifact
    print("\nStep 1: Saving PDF as artifact...")
    saved_filename = save_pdf_artifact(pdf_path_or_uri, artifact_service, app_name, USER_ID, session_id)

    if not saved_filename:
        print("Failed to save PDF as artifact. Aborting summarization.")
        return

    # 2. Create the user prompt mentioning the artifact
    user_query = f"Please summarize the document named `{saved_filename}`."
    print(f"\nStep 2: User Query to Agent: '{user_query}'")

    # 3. Run the agent
    print("\nStep 3: Running the Summarizer Agent...")
    content = types.Content(role='user', parts=[types.Part(text=user_query)])
    final_response_text = "[Agent did not produce a final response text]"

    try:
        async for event in runner.run_async(user_id=USER_ID, session_id=session_id, new_message=content):
            # Basic flow tracing
            if event.get_function_calls():
                fc = event.get_function_calls()[0]
                print(f"  [Agent requested tool: {fc.name} with args: {fc.args}]")
            elif event.get_function_responses():
                fr = event.get_function_responses()[0]
                print(f"  [Agent received result for tool: {fr.name}] (Content omitted for brevity)")

            # Capture final response
            if event.is_final_response() and event.content and event.content.parts:
                 if event.content.parts[0].text:
                      final_response_text = event.content.parts[0].text
                 else:
                      final_response_text = "[Non-text final response received]"
                 break # Exit loop on final response

    except Exception as e:
        print(f"  [Error during agent run]: {e}")
        final_response_text = f"[Error occurred: {e}]"


    print(f"\nStep 4: Final Agent Response:\n{final_response_text}")
    print(f"--- Flow Finished for {artifact_service.__class__.__name__} ---")

# --- Main Execution Function ---
async def main():

    # === Example 1: Using InMemoryArtifactService with Local PDF ===
    if LOCAL_PDF_PATH and LOCAL_PDF_PATH != "/path/to/your/local/document.pdf" and pypdf:
        mem_app_name = f"{APP_NAME_BASE}_mem"
        mem_session_id = f"{USER_ID}_mem_sess_1"
        mem_artifact_service = InMemoryArtifactService()
        mem_session_service = InMemorySessionService()
        await run_pdf_summarization(
            mem_artifact_service,
            mem_session_service,
            mem_app_name,
            mem_session_id,
            LOCAL_PDF_PATH
        )
    else:
        print("\nSkipping InMemoryArtifactService example: LOCAL_PDF_PATH not set or pypdf not available.")

    # === Example 2: Using GcsArtifactService with GCS PDF ===
    if GCS_PDF_URI and GCS_PDF_URI != "gs://your-gcs-bucket-name/path/to/your/document.pdf":
        gcs_app_name = f"{APP_NAME_BASE}_gcs"
        gcs_session_id = f"{USER_ID}_gcs_sess_1"
        gcs_artifact_service = GcsArtifactService(bucket_name=GCS_BUCKET_NAME)
        gcs_session_service = InMemorySessionService() # Session state still in memory
        await run_pdf_summarization(
            gcs_artifact_service,
            gcs_session_service,
            gcs_app_name,
            gcs_session_id,
            GCS_PDF_URI
        )
    else:
        print("\nSkipping GcsArtifactService example: GCS_PDF_URI not set.")


# --- Run the Examples ---
try:
    # Get/create event loop for Colab/script compatibility
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

loop.run_until_complete(main())

## Non-LLM Agents

## Multi-Agent - SequenceAgent

### Simple sequence flow; Parent->Child(2)

In [ ]:
import asyncio
from uuid import uuid4
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.invocation_context import InvocationContext, new_invocation_context_id
from google.adk.events import Event
from typing_extensions import override
from google.adk.sessions.in_memory_session_service import InMemorySessionService
from google.adk.sessions.session import Session
from google.genai import types
from typing import AsyncGenerator

class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )

class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
        )

async def main():

    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the SequentialAgent
    sequential_agent = SequentialAgent(name="SequentialAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=sequential_agent,
        user_content=types.Content(
                parts=[types.Part(text="execute")]
            )
        )

    # Run the SequentialAgent
    async for event in sequential_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")

await main()

Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...


### Simple sequence flow; Parent->Child(2) - without InnvocationContext

### Passing state between Children

In [ ]:
import asyncio
from uuid import uuid4
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.invocation_context import InvocationContext, new_invocation_context_id
from google.adk.events import Event
from typing_extensions import override
from google.adk.sessions.in_memory_session_service import InMemorySessionService
from google.adk.sessions.session import Session
from google.genai import types
from typing import AsyncGenerator

class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        # Set a value in the session state
        ctx.session.state["agent_a_value"] = "Hello from Agent A!"
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )

class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        # Retrieve the value from the session state
        agent_a_value = ctx.session.state.get("agent_a_value")
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text=f"Agent B: Received value: {agent_a_value}")]
            ),
        )
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
        )

async def main():

    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the SequentialAgent
    sequential_agent = SequentialAgent(name="SequentialAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=sequential_agent,
        user_content=types.Content(
                parts=[types.Part(text="execute")]
            )

        )

    # Run the SequentialAgent
    async for event in sequential_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")

await main()

Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Received value: Hello from Agent A!
Event: AgentB: Agent B: Finishing...


### Sequence with LLMAgent

In [ ]:
import asyncio
from uuid import uuid4
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.invocation_context import InvocationContext, new_invocation_context_id
from google.adk.events import Event
from typing_extensions import override
from google.adk.sessions.in_memory_session_service import InMemorySessionService
from google.adk.sessions.session import Session
from google.genai import types
from typing import AsyncGenerator
from google.adk.agents.llm_agent import LlmAgent

GEMINI_2_FLASH = "gemini-2.0-flash-001"

async def main():

    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = LlmAgent(name="AgentA", model = GEMINI_2_FLASH,
                       instruction = "You are Agent A. Respond with 'Agent A: Starting...'")
    agent_b = LlmAgent(name="AgentB", model = GEMINI_2_FLASH,
                       instruction = "You are Agent B. Respond with 'Agent B: Starting...'")

    # Create the SequentialAgent
    sequential_agent = SequentialAgent(name="SequentialAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=sequential_agent,
        user_content=types.Content(
                parts=[types.Part(text="execute")]
            )

        )

    # Run the SequentialAgent
    async for event in sequential_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")

await main()

Event: AgentA: Agent A: Starting...

Event: AgentB: Agent B: Starting...



### Sequence with LLMAgent and `runner.run_async`

In [ ]:
GEMINI_2_FLASH = "gemini-2.0-flash-001"

async def main():

    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user",
        session_id="1234"
    )

    agent_a = LlmAgent(name="AgentA", model = GEMINI_2_FLASH,
                       instruction = "You are Agent A. Respond with 'Agent A: Starting...'")
    agent_b = LlmAgent(name="AgentB", model = GEMINI_2_FLASH,
                       instruction = "You are Agent B. Respond with 'Agent B: Starting...'")

    # Create the SequentialAgent
    sequential_agent = SequentialAgent(name="SequentialAgent", children=[agent_a, agent_b])

    # Create the Runner
    runner = Runner(agent=sequential_agent, session_service=session_service, app_name="test_app")

    # Run the SequentialAgent through the Runner
    async for event in runner.run_async(user_id="test_user", session_id="1234", new_message=types.Content(role="user",
                parts=[types.Part(text="execute",
                                  )]
            )):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")

await main()

Event: AgentA: Agent A: Starting...

Event: AgentB: Agent B: Starting...
No previous requests to process. Ready for new instructions.



### Sequence with LLMAgent and simple runner

In [ ]:
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.llm_agent import LlmAgent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner


APP_NAME = "sequential_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "sequential_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"


agent_a = LlmAgent(
        name="AgentA",
        model=GEMINI_2_FLASH,
        instruction="You are Agent A. Respond with 'Agent A: Starting...'",
		output_key = "agent_a"
    )

agent_b = LlmAgent(
        name="AgentB",
        model=GEMINI_2_FLASH,
        instruction="You are Agent B. Respond with 'Agent B: Starting...'",
    )

# Create the SequentialAgent
sequential_agent = SequentialAgent(
        name="SequentialAgent", children=[agent_a, agent_b]
    )

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=sequential_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("execute")

Agent Response:  Agent A: Starting...

Agent Response:  Agent B: Starting...



### SequenceAgent + LLMAgent + States

In [ ]:
GEMINI_2_FLASH = "gemini-2.0-flash-001"

class SequentialAgent(BaseAgent):
    """A shell agent that run its child agents in sequence."""

    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        for child in self.children:
            async for event in child.run_async(ctx):
                yield event
                if event.actions and event.actions.state_delta:
                    ctx.session.state.update(event.actions.state_delta)

async def main():

    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = LlmAgent(name="AgentA", model = GEMINI_2_FLASH,
                       instruction = "You are Agent A. Respond with 'Agent Alpha1234: Starting...'",
                       output_key="agent_a_output")
    agent_b = LlmAgent(name="AgentB", model = GEMINI_2_FLASH,
                       instruction = "You are Agent B. Respond with 'Agent B: Starting...'",
                       )

    # Create the SequentialAgent
    sequential_agent = SequentialAgent(name="SequentialAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=sequential_agent,
        user_content=types.Content(
                parts=[types.Part(text="execute")]
            )

        )

    # Run the SequentialAgent
    async for event in sequential_agent.run_async(ctx):
        if event.content and event.content.parts:
            if event.author == "AgentB":
                agent_a_output = ctx.session.state.get("agent_a_output")
                print(f"Event: {event.author}: {event.content.parts[0].text} and Agent A output: {agent_a_output}")
            else:
                print(f"Event: {event.author}: {event.content.parts[0].text}")

await main()

Event: AgentA: Agent Alpha1234: Starting...

Event: AgentB: Agent B: Starting...
 and Agent A output: Agent Alpha1234: Starting...



## Multi-Agent - LoopAgent

### LoopAgent with Simple Runner

In [ ]:
from google.adk.agents.loop_agent import LoopAgent
from google.adk.agents.llm_agent import LlmAgent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner



APP_NAME = "loop_app"
USER_ID = "12345"
SESSION_ID = "123344"
AGENT_NAME = "loop_agent"
GEMINI_2_FLASH = "gemini-2.0-flash-001"




agent_a = LlmAgent(
        name="AgentA",
        model=GEMINI_2_FLASH,
        instruction="You are Agent A. Respond with 'Agent A: Starting...'",
    )

agent_b = LlmAgent(
        name="AgentB",
        model=GEMINI_2_FLASH,
        instruction="You are Agent B. Respond with 'Agent B: Starting...'",
    )

# Create the LoopAgent
loop_agent = LoopAgent(
    name="LoopAgent", children=[agent_a, agent_b], max_iterations=2
)


# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=loop_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("execute")

Agent Response:  Agent A: Starting...

Agent Response:  Agent B: Starting...

Agent Response:  Agent A: Acknowledged. Agent B is starting. I am ready for instructions.

Agent Response:  Agent B: Ready for instructions as well.



### LoopAgent with InnvocationContext

In [ ]:
from google.adk.agents.loop_agent import LoopAgent


class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )


class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
        )


async def main():
    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the LoopAgent
    loop_agent = LoopAgent(name="LoopAgent", children=[agent_a, agent_b], max_iterations=2)

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=loop_agent,
        user_content=types.Content(
            parts=[types.Part(text="execute")]
        ),
    )

    # Run the LoopAgent
    async for event in loop_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")


await main()

Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...
Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...


### Stop Condition with EventActions=Escalation

In [ ]:
from google.adk.agents.loop_agent import LoopAgent
from google.adk.events.event_actions import EventActions

class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )


class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
            actions=EventActions(escalate=True),
        )



async def main():
    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the LoopAgent
    loop_agent = LoopAgent(name="LoopAgent", children=[agent_a, agent_b], max_iterations=10)

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=loop_agent,
        user_content=types.Content(
            parts=[types.Part(text="execute")]
        ),
    )

    # Run the LoopAgent
    async for event in loop_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")


await main()

Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...


### Escalation with Condition - defined in state

In [ ]:
class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )


class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )

        # Example condition: Escalate if session state has a key 'escalate_agent_b'
        escalate = ctx.session.state.get('escalate_agent_b', False)

        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
            actions=EventActions(escalate=escalate),
        )


async def main():
    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the LoopAgent
    loop_agent = LoopAgent(name="LoopAgent", children=[agent_a, agent_b], max_iterations=3)

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=loop_agent,
        user_content=types.Content(
            parts=[types.Part(text="execute")]
        ),
    )

    # Example: Set a condition to escalate
    ctx.session.state['escalate_agent_b'] = False

    # Run the LoopAgent
    async for event in loop_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")


await main()

Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...
Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...
Event: AgentA: Agent A: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Starting...
Event: AgentB: Agent B: Finishing...


## ParallelAgent

### Simple

In [ ]:
from google.adk.agents.parallel_agent import ParallelAgent


class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        await asyncio.sleep(1)
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )


class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        await asyncio.sleep(2)
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Finishing...")]
            ),
        )


async def main():
    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the ParallelAgent
    parallel_agent = ParallelAgent(name="ParallelAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=parallel_agent,
        user_content=types.Content(
            parts=[types.Part(text="execute")]
        ),
    )

    # Run the ParallelAgent
    async for event in parallel_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")


await main()

Event: AgentA: Agent A: Starting...
Event: AgentB: Agent B: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Finishing...


### Shared State

In [ ]:
class AgentA(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Starting...")]
            ),
        )
        await asyncio.sleep(1)
        ctx.session.state['shared_data'] = "Data from Agent A"
        yield Event(
            author="AgentA",
            content=types.Content(
                parts=[types.Part(text="Agent A: Finishing...")]
            ),
        )


class AgentB(BaseAgent):
    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text="Agent B: Starting...")]
            ),
        )
        await asyncio.sleep(2)
        shared_data = ctx.session.state.get('shared_data', "No data from Agent A")
        yield Event(
            author="AgentB",
            content=types.Content(
                parts=[types.Part(text=f"Agent B: Finishing... Received: {shared_data}")]
            ),
        )


async def main():
    # Create a session
    session_service = InMemorySessionService()
    session: Session = session_service.create_session(
        app_name="test_app", user_id="test_user"
    )

    agent_a = AgentA(name="AgentA")
    agent_b = AgentB(name="AgentB")

    # Create the ParallelAgent
    parallel_agent = ParallelAgent(name="ParallelAgent", children=[agent_a, agent_b])

    # Create InvocationContext
    ctx = InvocationContext(
        invocation_id=new_invocation_context_id(),
        session_service=session_service,
        session=session,
        agent=parallel_agent,
        user_content=types.Content(
            parts=[types.Part(text="execute")]
        ),
    )

    # Run the ParallelAgent
    async for event in parallel_agent.run_async(ctx):
        if event.content and event.content.parts:
            print(f"Event: {event.author}: {event.content.parts[0].text}")


await main()

Event: AgentA: Agent A: Starting...
Event: AgentB: Agent B: Starting...
Event: AgentA: Agent A: Finishing...
Event: AgentB: Agent B: Finishing... Received: Data from Agent A


## CustomAgent

In [ ]:
class CustomAgent(BaseAgent):
    """A custom agent that generates a simple text message."""

    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        yield Event(
            invocation_id=ctx.invocation_id,
            author=self.name,
            content=types.Content(
                parts=[types.Part.from_text(text="Hello from Custom Agent!")],
            ),
        )


async def main():
    # Create a custom agent instance
    custom_agent = CustomAgent(name="CustomAgent", description="A custom agent")

    # Create a session service
    session_service = InMemorySessionService()

    # Create a session
    session = session_service.create_session(
        app_name="demo_app", user_id="test_user", session_id="test_session"
    )

    # Create a runner instance
    runner = Runner(
        app_name="demo_app",
        agent=custom_agent,
        session_service=session_service,
    )

    # Run the agent
    async for event in runner.run_async(
        user_id="test_user",
        session_id="test_session",
        new_message=types.Content(
            parts=[types.Part.from_text(text="Hi, how are you?")]
        ),
    ):
        print(f"Event: {event.content.parts[0].text}")


await main()

Event: Hello from Custom Agent!


### StoryFlowAgent Example

In [ ]:
import asyncio
from uuid import uuid4
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.invocation_context import InvocationContext, new_invocation_context_id
from google.adk.events import Event
from typing_extensions import override
from google.adk.sessions.in_memory_session_service import InMemorySessionService
from google.adk.sessions.session import Session
from google.genai import types
from typing import AsyncGenerator
from google.adk.agents.llm_agent import LlmAgent

class StoryFlowAgent(BaseAgent):
    """
    Custom agent for a story generation and refinement workflow.

    This agent orchestrates a sequence of LLM agents to generate a story,
    critique it, revise it, check grammar and tone, and potentially
    regenerate the story if the tone is negative.
    """

    def __init__(
        self,
        name: str,
        story_generator: LlmAgent,
        critic: LlmAgent,
        reviser: LlmAgent,
        grammar_check: LlmAgent,
        tone_check: LlmAgent,
    ):
        """
        Initializes the StoryFlowAgent.

        Args:
            name: The name of the agent.
            story_generator: An LlmAgent to generate the initial story.
            critic: An LlmAgent to critique the story.
            reviser: An LlmAgent to revise the story based on criticism.
            grammar_check: An LlmAgent to check the grammar.
            tone_check: An LlmAgent to analyze the tone.
        """
        super().__init__(name=name)  # Initialize the base class
        self.story_generator = story_generator
        self.critic = critic
        self.reviser = reviser
        self.grammar_check = grammar_check
        self.tone_check = tone_check

        # Create a LoopAgent for the critic-reviser cycle
        self.loop_agent = LoopAgent(
            name="CriticReviserLoop", children=[self.critic, self.reviser], max_iterations=2
        )

        # Create a SequentialAgent for post-processing
        self.sequential_agent = SequentialAgent(
            name="PostProcessing", children=[self.grammar_check, self.tone_check]
        )

        # Important: Define the child agents for framework management.
        self.children = [
            self.story_generator,
            self.loop_agent,
            self.sequential_agent,
        ]

    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        """
        Implements the custom orchestration logic for the story workflow.

        This method is an asynchronous generator that yields Event objects.
        It controls the execution of the child agents and implements the
        conditional regeneration based on tone.

        Args:
            ctx: The InvocationContext, providing access to session state.

        Yields:
            Event: Events generated by the child agents.
        """
        # 1. Initial Story Generation
        async for event in self.story_generator.run_async(ctx):
            yield event
            if event.is_final_response():
                story_content = event.content.model_dump_json()
                ctx.session.state["current_story"] = story_content

        # 2. Critic-Reviser Loop
        async for event in self.loop_agent.run_async(ctx):
            yield event
            # Assuming Reviser updates 'current_story' in session state.

        # 3. Sequential Post-Processing (Grammar and Tone Check)
        async for event in self.sequential_agent.run_async(ctx):
            yield event

        # 4. Tone-Based Conditional Logic
        tone_check_result = ctx.session.state.get("tone_check_result")
        if tone_check_result == "negative":
            # Regenerate story if the tone is negative
            ctx.session.state["current_story"] = ""  # Clear previous story.
            async for event in self.story_generator.run_async(ctx):
                yield event
        else:
            # If tone is not negative, do nothing (keep current story).
            pass



In [ ]:
# --- Define the individual LLM agents ---

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-1.5-flash",
    instruction="""You are a story writer. Write a short story about a cat,
    based on the topic provided in session state with key 'topic'""",
    input_schema=None,  # No specific input schema required
    output_key="current_story",  # Key for storing output in session state
)

critic = LlmAgent(
    name="Critic",
    model="gemini-1.5-flash",
    instruction="""You are a story critic.  Review the story provided in
    session state with key 'current_story'. Provide constructive criticism
    on how to improve it.""",
    input_schema=None,
    output_key="criticism",  # Key for storing criticism in session state
)

reviser = LlmAgent(
    name="Reviser",
    model="gemini-1.5-flash",
    instruction="""You are a story reviser.  Revise the story provided in
    session state with key 'current_story', based on the criticism in
    session state with key 'criticism'. Output the revised story.""",
    input_schema=None,
    output_key="current_story",  # Overwrites the original story
)

grammar_check = LlmAgent(
    name="GrammarCheck",
    model="gemini-1.5-flash",
    instruction="""You are a grammar checker. Check the grammar of the story
    provided in session state with key 'current_story'. Output any suggested
    corrections. If no corrections, output 'Grammar is good!'.""",
    input_schema=None,
    output_key="grammar_suggestions",
)

tone_check = LlmAgent(
    name="ToneCheck",
    model="gemini-1.5-flash",
    instruction="""You are a tone analyzer. Analyze the tone of the story
    provided in session state with key 'current_story'. Output 'positive' if
    the tone is positive, 'negative' if the tone is negative, or 'neutral'
    otherwise.""",
    input_schema=None,
    output_key="tone_check_result",
)


In [ ]:
# --- Create the custom agent instance ---
story_flow_agent = StoryFlowAgent(
    name="StoryFlowAgent",
    story_generator=story_generator,
    critic=critic,
    reviser=reviser,
    grammar_check=grammar_check,
    tone_check=tone_check,
)

ValueError: "StoryFlowAgent" object has no field "story_generator"

In [ ]:
import asyncio
import os
import uuid
from typing import Any, AsyncGenerator, Optional

# --- ADK Imports ---
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.agents.loop_agent import LoopAgent
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.invocation_context import InvocationContext, new_invocation_context_id
from google.adk.events import Event
from google.adk.sessions import InMemorySessionService, Session, State
from google.adk.runners import Runner
from google.genai import types
# *** Import PrivateAttr ***
from pydantic import Field, PrivateAttr
from typing_extensions import override

# --- Assume environment setup (like PROJECT_ID, LOCATION) is done ---
# print("Ensure GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION are set.")
# print("Ensure GOOGLE_GENAI_USE_VERTEXAI='TRUE' is set for Vertex AI.")
# os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"

# --- Constants ---
MODEL = "gemini-1.5-flash-latest" # Use a recent flash model
APP_NAME = "story_workflow_app"
USER_ID = "story_user_1"
SESSION_ID = f"story_session_{uuid.uuid4()}"

# --- Custom Agent Definition ---
class StoryFlowAgent(BaseAgent):
    """
    Custom agent orchestrating a story generation/refinement workflow.
    Demonstrates controlling child agents (LlmAgent, LoopAgent, SequentialAgent)
    from within the parent's _run_async_impl.
    Uses PrivateAttr for internally managed composite agents.
    """
    # --- Declare child agents as Pydantic fields ---
    story_generator: LlmAgent
    critic: LlmAgent
    reviser: LlmAgent
    grammar_check: LlmAgent
    tone_check: LlmAgent

    # --- Use PrivateAttr for internal composite agents ---
    # These are initialized in model_post_init and ignored by Pydantic validation.
    # Use leading underscore convention for private attributes.
    _loop_agent: LoopAgent = PrivateAttr()
    _sequential_agent: SequentialAgent = PrivateAttr()

    # --- Remove custom __init__ ---

    def model_post_init(self, __context: Any) -> None:
        """
        Runs after Pydantic initializes declared fields.
        Used here to construct composite agents and set up children.
        """
        super().model_post_init(__context) # Call parent's post_init

        # Create the LoopAgent using the initialized fields
        self._loop_agent = LoopAgent(
            name="CriticReviserLoop",
            children=[self.critic, self.reviser], # Use fields initialized by Pydantic
            max_iterations=1 # Keep loop short for demo
        )

        # Create the SequentialAgent
        self._sequential_agent = SequentialAgent(
            name="PostProcessing",
            children=[self.grammar_check, self.tone_check] # Use fields initialized by Pydantic
        )

        # Define the children list for the framework (important!)
        # BaseAgent's model_post_init handles setting parent links for these children.
        self.children = [
            self.story_generator,
            self._loop_agent,       # Use the private attribute
            self._sequential_agent, # Use the private attribute
        ]
        # Ensure parent links are set for the privately managed agents too
        self._loop_agent.parent_agent = self
        self._sequential_agent.parent_agent = self

        print(f"Initialized {self.name} with children: {[c.name for c in self.children]}")


    @override
    async def _run_async_impl(
        self, ctx: InvocationContext
    ) -> AsyncGenerator[Event, None]:
        """
        Implements the custom orchestration logic for the story workflow.
        Controls execution of child agents and implements conditional logic.
        """
        print(f"\n--- [{self.name}] Starting Workflow ---")
        initial_topic = ctx.state.get('topic', 'a brave mouse')
        print(f"--- [{self.name}] Topic from state: '{initial_topic}' ---")

        # 1. Initial Story Generation
        print(f"\n--- [{self.name}] Running StoryGenerator ---")
        async for event in self.story_generator.run_async(ctx):
            yield event
        # story_generator's output_key="current_story" handles saving

        print(f"\n--- [{self.name}] Current story (after generate): '{ctx.state.get('current_story', 'NOT SET')[:50]}...' ---")

        # 2. Critic-Reviser Loop (use the private attribute)
        print(f"\n--- [{self.name}] Running CriticReviserLoop ---")
        async for event in self._loop_agent.run_async(ctx):
            yield event
        # Reviser's output_key="current_story" handles saving the revision

        print(f"\n--- [{self.name}] Current story (after loop): '{ctx.state.get('current_story', 'NOT SET')[:50]}...' ---")
        print(f"--- [{self.name}] Criticism was: '{ctx.state.get('criticism', 'NOT SET')[:50]}...' ---")


        # 3. Sequential Post-Processing (use the private attribute)
        print(f"\n--- [{self.name}] Running PostProcessing (Grammar/Tone) ---")
        async for event in self._sequential_agent.run_async(ctx):
            yield event
        # grammar_check and tone_check save their results via output_key

        print(f"\n--- [{self.name}] Grammar suggestions: '{ctx.state.get('grammar_suggestions', 'NOT SET')[:50]}...' ---")

        # 4. Tone-Based Conditional Logic
        print(f"\n--- [{self.name}] Checking Tone ---")
        # Access state via ctx.state
        tone_check_result = ctx.state.get("tone_check_result")
        print(f"--- [{self.name}] Tone result from state: {tone_check_result} ---")

        # Check if the tone string contains 'negative' (case-insensitive)
        is_negative = isinstance(tone_check_result, str) and "negative" in tone_check_result.lower()

        if is_negative:
            print(f"--- [{self.name}] Tone is negative. Regenerating story... ---")
            # Regenerate story
            # Story_generator will overwrite state['current_story'] via output_key
            async for event in self.story_generator.run_async(ctx):
                yield event
            print(f"\n--- [{self.name}] Regenerated story: '{ctx.state.get('current_story', 'NOT SET')[:50]}...' ---")

        else:
            print(f"--- [{self.name}] Tone is acceptable ('{tone_check_result}'). Workflow complete. ---")

        print(f"\n--- [{self.name}] Finishing Workflow ---")


# --- Define the individual LLM agents (as before) ---

story_generator = LlmAgent(
    name="StoryGenerator", model=MODEL,
    instruction="You are a story writer. Write a short story (2-3 sentences) about the topic provided in session state key 'topic'.",
    output_key="current_story",
)

critic = LlmAgent(
    name="Critic", model=MODEL,
    instruction="You are a story critic. Review the story in session state key 'current_story'. Provide one sentence of constructive criticism.",
    output_key="criticism",
)

reviser = LlmAgent(
    name="Reviser", model=MODEL,
    instruction="You are a story reviser. Revise the story in session state key 'current_story' based on the criticism in session state key 'criticism'. Output only the revised story (2-3 sentences).",
    output_key="current_story", # Overwrites original/previous revision
)

grammar_check = LlmAgent(
    name="GrammarCheck", model=MODEL,
    instruction="You are a grammar checker. Check the grammar of the story in session state key 'current_story'. Output only suggested corrections (max 1 sentence). If no corrections, output 'Grammar looks good!'.",
    output_key="grammar_suggestions",
)

tone_check = LlmAgent(
    name="ToneCheck", model=MODEL,
    instruction="You are a tone analyzer. Analyze the tone of the story in session state key 'current_story'. Output ONLY one word: 'positive', 'negative', or 'neutral'.",
    output_key="tone_check_result",
)

# --- Main execution block ---
async def main():
    print("--- Setting up Story Workflow ---")
    # 1. Create the custom agent instance
    # Pydantic automatically calls model_post_init after initializing the fields
    story_flow_agent = StoryFlowAgent(
        name="StoryFlowAgent",
        story_generator=story_generator,
        critic=critic,
        reviser=reviser,
        grammar_check=grammar_check,
        tone_check=tone_check,
    )

    # 2. Setup Session and Runner
    session_service = InMemorySessionService()
    initial_topic = "a shy squirrel making a friend" # Changed topic slightly
    session = session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={"topic": initial_topic} # Provide initial topic
    )
    runner = Runner(
        agent=story_flow_agent, # Use the custom orchestrator agent
        app_name=APP_NAME,
        session_service=session_service
    )
    print(f"Session created (ID: {session.id}) with initial topic: '{initial_topic}'")

    # 3. Run the workflow
    print("\n--- Starting Agent Run ---")
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.Content(parts=[types.Part(text="Start story workflow.")]) # Dummy message to trigger run
    ):
        # Print events from child agents as they happen
        if event.content and event.content.parts:
             # Simple check to avoid printing empty/non-text parts if any
             text_content = event.content.parts[0].text if event.content.parts[0].text else "[Non-text content]"
             print(f"  Event from [{event.author}]: {text_content.strip()}")
        elif event.actions.state_delta and event.author != story_flow_agent.name: # Avoid printing empty state deltas from parent
             # Filter empty deltas unless you specifically want to see them
             if event.actions.state_delta:
                  print(f"  State Delta from [{event.author}]: {event.actions.state_delta}")


    # 4. Show final state
    print("\n--- Workflow Finished. Final Session State: ---")
    final_session = session_service.get_session(APP_NAME, USER_ID, SESSION_ID)
    final_story = final_session.state.get('current_story', 'N/A')
    final_tone = final_session.state.get('tone_check_result', 'N/A')
    final_grammar = final_session.state.get('grammar_suggestions', 'N/A')
    print(f"  Topic: {final_session.state.get('topic', 'N/A')}")
    print(f"  Final Story: {final_story}")
    print(f"  Criticism: {final_session.state.get('criticism', 'N/A')}")
    print(f"  Grammar Suggs: {final_grammar}")
    print(f"  Final Tone: {final_tone}")
    # print(f" Full State: {final_session.state}") # Optional: print entire state

# --- Run the main function ---
await main()

--- Setting up Story Workflow ---
Initialized StoryFlowAgent with children: ['StoryGenerator', 'CriticReviserLoop', 'PostProcessing']
Session created (ID: story_session_175121c4-3ca8-4b5e-9cfa-a6c459c39837) with initial topic: 'a shy squirrel making a friend'

--- Starting Agent Run ---

--- [StoryFlowAgent] Starting Workflow ---


AttributeError: 'InvocationContext' object has no attribute 'state'